# Testing Molly's code

6/5/2024 | ZG

Goals:
1) Visualize runoff predicted by all RCPs using one GCM
2) Try another GCM
3) Plot annual runoff change

# Step 1: visualize runoff predicted by all RCPs using NorESM1-M

Things to figure out: 
- No need to avg GCM values, only using one
- downloading data

In [1]:
# Import packages
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import pandas as pd
import seaborn as sns
import os

### Functions

In [2]:
# # Function: load the dataset
# def open_series(fpath_stem, which_date, which_gcm, which_rcp):
#     this_fpath = fpath_stem+'{}-gcm_data_{}_{}_compiled_output.nc'.format(which_date, which_gcm, which_rcp)
#     this_ds = xr.open_dataset(this_fpath)
#     return this_ds

In [3]:
# Function: load the dataset
def open_series(fpath_stem, which_date, which_gcm, which_ssp):
    # Update the file path format to match the changes in the previous cell
    this_fpath = os.path.expanduser(fpath_stem + '{}-gcm_data_{}_{}_compiled_output.nc'.format(which_date, which_gcm, which_ssp))
    this_ds = xr.open_dataset(this_fpath)
    return this_ds

In [4]:
# Function: calculate runoff
def calc_seasonal(this_ds): # do everything below without the function
    melt1 = this_ds.melt_on_glacier_monthly
    melt2 = this_ds.melt_off_glacier_monthly
    melt3 = this_ds.liq_prcp_off_glacier_monthly
    melt4 = this_ds.liq_prcp_on_glacier_monthly
    
    this_sum = melt1.sum(dim='rgi_id').dropna(dim='time') + melt2.sum(dim='rgi_id').dropna(dim='time') + melt3.sum(dim='rgi_id').dropna(dim='time') + melt4.sum(dim='rgi_id').dropna(dim='time')
    nonzero_sum = this_sum.where(this_sum!=0, drop=True)
    nonzero_sum *= 1e-9


    return nonzero_sum

In [5]:
# Function: select a year and take the mean runoff of all gcms for this year
def season_sliced(keys, dictionary, time):
    new_df = pd.DataFrame()
    for k in keys:
        new = dictionary[k]
        new = new[time]
        new = pd.DataFrame(new)
        new_df[k] = new
    new_df['mean'] = new_df.transpose().mean()
    return new_df

In [6]:
# Make a function that calculates a given runoff value, which I can visualize later
def rtype_calc_monthly(keys, dictionary, runoff):
    dataframe = pd.DataFrame()
    for k in keys:
        rtype_gcm = dictionary[k]
        rtype = rtype_gcm[runoff]
        rtype = pd.DataFrame(rtype)
        dataframe[k] = rtype
    dataframe['mean'] = dataframe.transpose().mean()
    return dataframe

In [7]:
# Make a function that calculates a given runoff value, which I can visualize later
def rtype_calc_monthly_2vars(keys, dictionary, runoff1, runoff2):
    dataframe = pd.DataFrame()
    for k in keys:
        rtype_gcm = dictionary[k]
        rtype1 = rtype_gcm[runoff1]
        rtype2 = rtype_gcm[runoff2]
        rsum = rtype1 + rtype2
        rtype_sum = pd.DataFrame(rsum)
        dataframe[k] = rtype_sum
    dataframe['mean'] = dataframe.transpose().mean()
    return dataframe

In [8]:
# Define variables that will be used for each test
base = '/Users/ziggygoddard/Documents/summer24/'

gcm_list = ['mri-esm2-0_r1i1p1f1'] 

date_list = ['2024-06-12'] # export date (date I downloaded the data)

ssp_list = ['ssp126', 'ssp370','ssp585'] # this stays the same

gcm_w60 = ['mri-esm2-0_r1i1p1f1'] # not sure what gcm_w60 is/if it should be called something else

all_monthly_dict = {}

# These have to be files that I have downloaded

## Try to make a mega-dictionary with all RCPs

In [9]:
# create dictionary
all_monthly_dict = {}

In [10]:
x=0
ssp = ['ssp126', 'ssp370','ssp585']

# Iterate over all combinations of GCMs, dates, and RCP scenarios
# "r" switched to "s"
for gcm in gcm_list:
    for date in date_list:
        for s in ssp_list:
            # Generate the file path
            filepath = base + '{}-gcm_data_{}_{}_compiled_output.nc'.format(date, gcm, s)
            # Check if the file exists
            if os.path.exists(filepath):
                # Open dataset and store in dictionary
                ssp_value = open_series(fpath_stem=base, which_date=date, which_gcm=gcm, which_ssp=s)
                k = '{}-{}'.format(gcm, s)
                all_monthly_dict[k] = ssp_value

In [11]:
# x=0
# rcp = ['rcp26', 'rcp45', 'rcp60', 'rcp85']

# for r in rcp:
#     x = 0
#     for gcm in gcm_list:
#         if r != 'rcp60':
#             rcp_value = open_series(fpath_stem=base, which_date=date_list[x], which_gcm=gcm, which_rcp=r)
#             k = '{}-{}'.format(gcm, r)
#             all_monthly_dict[k] = rcp_value
#         else:
#             if gcm in gcm_w60:
#                 rcp_value = open_series(fpath_stem=base, which_date=date_list[x], which_gcm=gcm, which_rcp=r)
#                 k = '{}-{}'.format(gcm, r)
#                 all_monthly_dict[k]=rcp_value
#         x += 1

In [12]:
x=0
ssp = ['ssp126', 'ssp370','ssp585']

for s in ssp:
    x = 0
    for gcm in gcm_list:
        ssp_value = open_series(fpath_stem=base, which_date=date_list[x], which_gcm=gcm, which_ssp=s)
        k = '{}-{}'.format(gcm, s)
        all_monthly_dict[k] = ssp_value
        
    x += 1

In [13]:
# Define keys
keys_all = all_monthly_dict.keys()

In [14]:
all_unrolled = {}
for k in keys_all:
    calculation = calc_seasonal(all_monthly_dict[k])
    z = '{}'.format(k)
    all_unrolled[z] = calculation

Roll values for plotting

In [15]:
n_roll = 3 # this is the roll value for all the plots

dict_roll_all = {}
dict_plot_all = {}

for k in keys_all:
    season_roll = all_unrolled[k].roll(month_2d=n_roll, roll_coords=True) # check about these
    season_forplot = all_unrolled[k].roll(month_2d=n_roll, roll_coords=True) 
    name = '{}'.format(k)
    dict_roll_all[name] = season_roll
    dict_plot_all[name] = season_forplot

In [16]:
dict_roll_all

{'mri-esm2-0_r1i1p1f1-ssp126': <xarray.DataArray (time: 81, month_2d: 12)> Size: 4kB
 array([[8.99941063e+00, 1.20390282e+01, 1.66127663e+01, 1.80041599e+01,
         1.46102371e+01, 8.81342602e+00, 5.56925058e+00, 2.15781522e+00,
         1.01832680e-01, 1.49818838e-01, 1.96281210e-01, 4.19290590e+00],
        [1.01367359e+01, 1.11781597e+01, 1.18271027e+01, 1.40965834e+01,
         1.33349104e+01, 7.91780710e+00, 6.18942547e+00, 1.42081499e+00,
         9.16064158e-02, 4.02938724e-02, 2.14820892e-01, 3.30335903e+00],
        [4.95397377e+00, 1.08127079e+01, 1.47945280e+01, 1.22305861e+01,
         1.08411798e+01, 1.04562578e+01, 3.76884031e+00, 4.83149618e-01,
         4.35064062e-02, 6.33053631e-02, 9.71728742e-01, 3.61162186e+00],
        [1.10364933e+01, 1.25111513e+01, 1.35994158e+01, 9.40830803e+00,
         7.94512272e+00, 8.53127003e+00, 5.72765446e+00, 4.05501306e-01,
         2.48465478e-01, 1.53422907e-01, 1.07171130e+00, 5.34574175e+00],
        [1.11422615e+01, 1.34307642

Take sum over rgi id

In [17]:
# Roll and sum
rtype_all = {}
#n_roll = o # this isn't necessary anymore

for k in keys_all:
    rtype_season = all_monthly_dict[k].roll(roll_coords=True)
    rtype_sum = rtype_season.sum(dim='rgi_id')
    rtype_name = '{}'.format(k)
    rtype_all[rtype_name] = rtype_sum

In [18]:
# # Not sur how to alter this one either
# # Make a dictionary with all data for all GCMs for each RCP
# # Modified - ensures that only valid file paths are attempted to be opened
# rcp_monthly_dict = {}

# for rcp in rcp_list:
#     rcp_dict = {}
#     for gcm in gcm_list:
#         for date in date_list:
#             # Check if the RCP is not 'rcp60' or the GCM is in gcm_w60
#             if rcp != 'rcp60' or gcm in gcm_w60:
#                 filepath = base + '{}-gcm_data_{}_{}_compiled_output.nc'.format(date, gcm, rcp)
#                 # Check if the file exists before attempting to open it
#                 if os.path.exists(filepath):
#                     rcp_value = open_series(fpath_stem=base, which_date=date, which_gcm=gcm, which_rcp=rcp)
#                     k = '{}-{}'.format(gcm, rcp)
#                     rcp_dict[k] = rcp_value
#     rcp_monthly_dict[rcp] = rcp_dict

In [19]:
# Not sur how to alter this one either
# Make a dictionary with all data for all GCMs for each RCP
# Modified - ensures that only valid file paths are attempted to be opened
ssp_monthly_dict = {}

for ssp in ssp_list:
    ssp_dict = {}
    for gcm in gcm_list:
        for date in date_list:
            filepath = base + '{}-gcm_data_{}_{}_compiled_output.nc'.format(date, gcm, ssp)
                # Check if the file exists before attempting to open it
            if os.path.exists(filepath):
                ssp_value = open_series(fpath_stem=base, which_date=date, which_gcm=gcm, which_ssp=ssp)
                k = '{}-{}'.format(gcm, ssp)
                ssp_dict[k] = ssp_value
    ssp_monthly_dict[ssp] = ssp_dict

### Calculate rcp26

In [20]:
# # Make a dictionary with all data for all GCMs (this will only output one rcp!)
# rcp_monthly_dict26 = {}

# x=0
# rcp = 'rcp26'

# # Per rcp:
# for gcm in gcm_list:
#     if rcp != 'rcp60':
#         rcp_value = open_series(fpath_stem=base, which_date=date_list[x], which_gcm=gcm, which_rcp=rcp)
#         k = '{}-{}'.format(gcm, rcp)
#         rcp_monthly_dict26[k]=rcp_value
#     else:
#         if gcm in gcm_w60:
#             rcp_value = open_series(fpath_stem=base, which_date=date_list[x], which_gcm=gcm, which_rcp=rcp)
#             k = '{}-{}'.format(gcm, rcp)
#             rcp_monthly_dict26[k]=rcp_value
#     x += 1

In [21]:
# Make a dictionary with all data for all GCMs (this will only output one rcp!)
ssp_monthly_dict126 = {}

x=0
ssp = 'ssp126'

# Per ssp:
for ssp in ssp_list:
    ssp_value = open_series(fpath_stem=base, which_date=date_list[x], which_gcm=gcm, which_ssp=ssp)
    k = '{}-{}'.format(gcm, ssp)
    ssp_monthly_dict126[k]=ssp_value
    x += 1

IndexError: list index out of range

In [ ]:
import os
import xarray as xr

# Function: load the dataset
def open_series(fpath_stem, which_date, which_gcm, which_ssp):
    this_fpath = os.path.expanduser(fpath_stem + '{}-gcm_data_{}_{}_compiled_output.nc'.format(which_date, which_gcm, which_ssp))
    if os.path.exists(this_fpath):
        this_ds = xr.open_dataset(this_fpath)
        return this_ds
    else:
        print(f"File {this_fpath} does not exist.")
        return None

# Base path
base = '/Users/ziggygoddard/Documents/summer24/'

# Lists
gcm_list = ['mri-esm2-0_r1i1p1f1']
date_list = ['2024-06-12']  # Ensure this date list is correct and matches your file names
ssp_list = ['ssp126', 'ssp370','ssp585']

# Dictionary to hold the data
ssp_monthly_dict = {}

# Per ssp:
for ssp in ssp_list:
    for i, date in enumerate(date_list):
        for gcm in gcm_list:
            # Generate the file path
            ssp_value = open_series(fpath_stem=base, which_date=date, which_gcm=gcm, which_ssp=ssp)
            if ssp_value is not None:
                k = '{}-{}'.format(gcm, ssp)
                ssp_monthly_dict[k] = ssp_value

# Checking the dictionary
print(ssp_monthly_dict.keys())


In [ ]:
# need to figure out why this is only one GCM (CCSM4 in the OG code)
# replace it with NorESM1-M
# this works!

rcp_monthly_dict26['NorESM1-M-rcp26']

In [ ]:
# Bring in your dictionary of unrolled values to find TOTAL RUNOFF (for time slice plot)
season_unroll26 = {}
for k in rcp_monthly_dict26.keys():
    calculation = calc_seasonal(rcp_monthly_dict26[k])
    z = '{}-calc'.format(k)
    season_unroll26[z] = calculation

In [ ]:
# make current gcm

season_unroll26['NorESM1-M-rcp26-calc']

In [ ]:
# define keys to use in next step
keys26 = season_unroll26.keys()

In [ ]:
# Roll coordinates!
## Let's try directly assigning the coordinate used for plotting?
#roll_value = -3 # we don't want to set roll values here
#n_roll = roll_value

dict_roll26 = {}
dict_plot26 = {}

for k in keys26:
    season_roll = season_unroll26[k].roll(month_2d=n_roll, roll_coords=False) # this has to be false
    season_forplot = season_unroll26[k].roll(month_2d=n_roll, roll_coords=False)
    name = '{}'.format(k)
    dict_roll26[name] = season_roll
    dict_plot26[name] = season_forplot

In [ ]:
# Roll and sum
rtype_roll26 = {}
#n_roll = -roll_value

for k in rcp_monthly_dict26.keys():
    rtype_season = rcp_monthly_dict26[k].roll(month_2d=n_roll, roll_coords=True)
    rtype_sum = rtype_season.sum(dim='rgi_id')
    rtype_name = '{}'.format(k)
    rtype_roll26[rtype_name] = rtype_sum

# Output is a dictionary with summed values!

In [ ]:
rtype_roll26.keys()

In [ ]:
rtype_calc_s26 = {}
rtype_calc_e26 = {}

for k in rtype_roll26.keys():
    df_key = rtype_roll26[k]
    df_m_s = df_key.isel(time=slice(0, 10)).mean(dim='time').to_dataframe() * 1e-9
    df_m_s.columns = [c.replace('_monthly', '_m') for c in df_m_s.columns]
    df_m_s = df_m_s.clip(0)
    calc_name = '{}'.format(k)
    rtype_calc_s26[calc_name] = df_m_s

for k in rtype_roll26.keys():
    df_key = rtype_roll26[k]
    df_m_e = df_key.isel(time=slice(-11, -1)).mean(dim='time').to_dataframe() * 1e-9
    df_m_e.columns = [c.replace('_monthly', '_m') for c in df_m_e.columns]
    df_m_e = df_m_e.clip(0)
    calc_name = '{}'.format(k)
    rtype_calc_e26[calc_name] = df_m_e

In [ ]:
# IN USE: This is used for visualization of the 6-rolled hydro months, turns it into calendar months w/shift
# MODIFICATION: originally, roll(month_2d=-3, roll_coords=True); however, in new version month_2d=0, so not necessary to include
season_roll2 = season_unroll26['NorESM1-M-rcp26-calc'].roll(month_2d=n_roll, roll_coords=True)

In [ ]:
# IN USE: This is used for plotting of months with values 1-12
index1 = season_unroll26['NorESM1-M-rcp26-calc']
index2 = index1['month_2d']

In [ ]:
# Make this into a visual
runoff_variables = ['melt_off_glacier_m', 'melt_on_glacier_m', 'liq_prcp_off_glacier_m', 'liq_prcp_on_glacier_m']
s_keys = rtype_calc_s26.keys()

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7), sharey=True);

for r in runoff_variables:
    runoff_s = rtype_calc_monthly(s_keys, rtype_calc_s26, r)
    runoff_e = rtype_calc_monthly(s_keys, rtype_calc_e26, r)
    xvals = index2
    ax1.plot(xvals, runoff_s['mean'], label=r)
    ax2.plot(xvals, runoff_e['mean'], label=r)
    
ax1.set(ylabel='Runoff [Mt]', title='2020-2030')
ax2.set(title='2090-2100')
# THIS WORKS
for ax in (ax1,ax2):
    ax.set(xticks = season_roll['month_2d'].values,
           xticklabels = season_roll2['month_2d'].values, #here, we want calendar year values
           xlabel = 'Calendar Month'
          );
    ax.legend();

plt.ylim([-0.5,15]);

Why do the graphs above start at month 10? They do in Molly's notebook as well (come back to this)

### Calculate rcp45

In [ ]:
# Make a dictionary with all data for all GCMs (this will only output one rcp!)
rcp_monthly_dict45 = {}

x=0
rcp = 'rcp45'

# Per rcp:
for gcm in gcm_list:
    if rcp != 'rcp60':
        rcp_value = open_series(fpath_stem=base, which_date=date_list[x], which_gcm=gcm, which_rcp=rcp)
        k = '{}-{}'.format(gcm, rcp)
        rcp_monthly_dict45[k]=rcp_value
    else:
        if gcm in gcm_w60:
            rcp_value = open_series(fpath_stem=base, which_date=date_list[x], which_gcm=gcm, which_rcp=rcp)
            k = '{}-{}'.format(gcm, rcp)
            rcp_monthly_dict45[k]=rcp_value
    x += 1

In [ ]:
# Bring in your dictionary of unrolled values
season_unroll45 = {}
for k in rcp_monthly_dict45.keys():
    calculation = calc_seasonal(rcp_monthly_dict45[k])
    z = '{}-calc'.format(k)
    season_unroll45[z] = calculation

In [ ]:
# define keys to use in next step
keys45 = season_unroll45.keys()

In [ ]:
# Roll coordinates!

dict_roll45 = {}
dict_plot45 = {}

for k in keys45:
    season_roll = season_unroll45[k].roll(month_2d=n_roll, roll_coords=False)
    season_forplot = season_unroll45[k].roll(month_2d=n_roll, roll_coords=False)
    name = '{}'.format(k)
    dict_roll45[name] = season_roll
    dict_plot45[name] = season_forplot

In [ ]:
# Roll and sum
rtype_roll45 = {}

for k in rcp_monthly_dict45.keys():
    rtype_season = rcp_monthly_dict45[k].roll(month_2d=n_roll, roll_coords=True)
    rtype_sum = rtype_season.sum(dim='rgi_id')
    rtype_name = '{}'.format(k)
    rtype_roll45[rtype_name] = rtype_sum

# Output is a dictionary with summed values!

In [ ]:
rtype_roll45.keys()

In [ ]:
rtype_calc_s45 = {}
rtype_calc_e45 = {}

for k in rtype_roll45.keys():
    df_key = rtype_roll45[k]
    df_m_s = df_key.isel(time=slice(0, 10)).mean(dim='time').to_dataframe() * 1e-9
    df_m_s.columns = [c.replace('_monthly', '_m') for c in df_m_s.columns]
    df_m_s = df_m_s.clip(0)
    calc_name = '{}'.format(k)
    rtype_calc_s45[calc_name] = df_m_s

for k in rtype_roll45.keys():
    df_key = rtype_roll45[k]
    df_m_e = df_key.isel(time=slice(-11, -1)).mean(dim='time').to_dataframe() * 1e-9
    df_m_e.columns = [c.replace('_monthly', '_m') for c in df_m_e.columns]
    df_m_e = df_m_e.clip(0)
    calc_name = '{}'.format(k)
    rtype_calc_e45[calc_name] = df_m_e
    

In [ ]:
# IN USE: This is used for visualization of the 6-rolled hydro months, turns it into calendar months w/shift
season_roll2 = season_unroll45['NorESM1-M-rcp45-calc'].roll(month_2d=n_roll, roll_coords=True)

In [ ]:
# IN USE: This is used for plotting of months with values 1-12
index1 = season_unroll45['NorESM1-M-rcp45-calc']
index2 = index1['month_2d']

In [ ]:
# Make this into a visual
runoff_variables = ['melt_off_glacier_m', 'melt_on_glacier_m', 'liq_prcp_off_glacier_m', 'liq_prcp_on_glacier_m']
s_keys = rtype_calc_s45.keys()

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7), sharey=True);

for r in runoff_variables:
    runoff_s = rtype_calc_monthly(s_keys, rtype_calc_s45, r)
    runoff_e = rtype_calc_monthly(s_keys, rtype_calc_e45, r)
    xvals = index2
    ax1.plot(xvals, runoff_s['mean'], label=r)
    ax2.plot(xvals, runoff_e['mean'], label=r)
    
ax1.set(ylabel='Runoff [Mt]', title='2020-2030')
ax2.set(title='2090-2100')
# THIS WORKS
for ax in (ax1,ax2):
    ax.set(xticks = season_roll['month_2d'].values,
           xticklabels = season_roll2['month_2d'].values, #here, we want calendar year values
           xlabel = 'Calendar Month'
          );
    ax.legend();
    
plt.ylim([-0.5,20]);

These plots look different than Molly's (later peak runoff)

### Calculate rcp60

In [ ]:
# Make a dictionary with all data for all GCMs (this will only output one rcp!)
rcp_monthly_dict60 = {}

x=0
rcp = 'rcp60'

# Per rcp:
for gcm in gcm_list:
    if rcp != 'rcp60':
        rcp_value = open_series(fpath_stem=base, which_date=date_list[x], which_gcm=gcm, which_rcp=rcp)
        k = '{}-{}'.format(gcm, rcp)
        rcp_monthly_dict60[k]=rcp_value
    else:
        if gcm in gcm_w60:
            rcp_value = open_series(fpath_stem=base, which_date=date_list[x], which_gcm=gcm, which_rcp=rcp)
            k = '{}-{}'.format(gcm, rcp)
            rcp_monthly_dict60[k]=rcp_value
    x += 1

In [ ]:
# Bring in your dictionary of unrolled values
season_unroll60 = {}
for k in rcp_monthly_dict60.keys():
    calculation = calc_seasonal(rcp_monthly_dict60[k])
    z = '{}-calc'.format(k)
    season_unroll60[z] = calculation

In [ ]:
# define keys to use in next step
keys60 = season_unroll60.keys()

In [ ]:
# Roll coordinates!

dict_roll60 = {}
dict_plot60 = {}

for k in keys60:
    season_roll = season_unroll60[k].roll(month_2d=n_roll, roll_coords=False)
    season_forplot = season_unroll60[k].roll(month_2d=n_roll, roll_coords=False)
    name = '{}'.format(k)
    dict_roll60[name] = season_roll
    dict_plot60[name] = season_forplot

In [ ]:
# Roll and sum
rtype_roll60 = {}

for k in rcp_monthly_dict60.keys():
    rtype_season = rcp_monthly_dict60[k].roll(month_2d=n_roll, roll_coords=True)
    rtype_sum = rtype_season.sum(dim='rgi_id')
    rtype_name = '{}'.format(k)
    rtype_roll60[rtype_name] = rtype_sum

# Output is a dictionary with summed values!

In [ ]:
rtype_roll60.keys()

In [ ]:
rtype_calc_s60 = {}
rtype_calc_e60 = {}

for k in rtype_roll60.keys():
    df_key = rtype_roll60[k]
    df_m_s = df_key.isel(time=slice(0, 10)).mean(dim='time').to_dataframe() * 1e-9
    df_m_s.columns = [c.replace('_monthly', '_m') for c in df_m_s.columns]
    df_m_s = df_m_s.clip(0)
    calc_name = '{}'.format(k)
    rtype_calc_s60[calc_name] = df_m_s

for k in rtype_roll60.keys():
    df_key = rtype_roll60[k]
    df_m_e = df_key.isel(time=slice(-11, -1)).mean(dim='time').to_dataframe() * 1e-9
    df_m_e.columns = [c.replace('_monthly', '_m') for c in df_m_e.columns]
    df_m_e = df_m_e.clip(0)
    calc_name = '{}'.format(k)
    rtype_calc_e60[calc_name] = df_m_e
    

In [ ]:
# IN USE: This is used for visualization of the 6-rolled hydro months, turns it into calendar months w/shift
season_roll2 = season_unroll60['NorESM1-M-rcp60-calc'].roll(month_2d=n_roll, roll_coords=True)

In [ ]:
# IN USE: This is used for plotting of months with values 1-12
index1 = season_unroll60['NorESM1-M-rcp60-calc']
index2 = index1['month_2d']

In [ ]:
# Make this into a visual
runoff_variables = ['melt_off_glacier_m', 'melt_on_glacier_m', 'liq_prcp_off_glacier_m', 'liq_prcp_on_glacier_m']
s_keys = rtype_calc_s60.keys()

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7), sharey=True);

for r in runoff_variables:
    runoff_s = rtype_calc_monthly(s_keys, rtype_calc_s60, r)
    runoff_e = rtype_calc_monthly(s_keys, rtype_calc_e60, r)
    xvals = index2
    ax1.plot(xvals, runoff_s['mean'], label=r)
    ax2.plot(xvals, runoff_e['mean'], label=r)

ax1.set(ylabel='Runoff [Mt]', title='2020-2030')
ax2.set(title='2090-2100')
# THIS WORKS
for ax in (ax1,ax2):
    ax.set(xticks = season_roll['month_2d'].values,
           xticklabels = season_roll2['month_2d'].values, #here, we want calendar year values
           xlabel = 'Calendar Month'
          );
    ax.legend();

plt.ylim([-0.5,20]);

### Calculate rcp85

In [ ]:
# Make a dictionary with all data for all GCMs (this will only output one rcp!)
rcp_monthly_dict85 = {}

x=0
rcp = 'rcp85'

# Per rcp:
for gcm in gcm_list:
    if rcp != 'rcp60':
        rcp_value = open_series(fpath_stem=base, which_date=date_list[x], which_gcm=gcm, which_rcp=rcp)
        k = '{}-{}'.format(gcm, rcp)
        rcp_monthly_dict85[k]=rcp_value
    else:
        if gcm in gcm_w60:
            rcp_value = open_series(fpath_stem=base, which_date=date_list[x], which_gcm=gcm, which_rcp=rcp)
            k = '{}-{}'.format(gcm, rcp)
            rcp_monthly_dict85[k]=rcp_value
    x += 1

In [ ]:
# Bring in your dictionary of unrolled values
season_unroll85 = {}
for k in rcp_monthly_dict85.keys():
    calculation = calc_seasonal(rcp_monthly_dict85[k])
    z = '{}-calc'.format(k)
    season_unroll85[z] = calculation

In [ ]:
# define keys to use in next step
keys85 = season_unroll85.keys()

In [ ]:
# Roll coordinates!

dict_roll85 = {}
dict_plot85 = {}

for k in keys85:
    season_roll = season_unroll85[k].roll(month_2d=n_roll, roll_coords=False)
    season_forplot = season_unroll85[k].roll(month_2d=n_roll, roll_coords=False)
    name = '{}'.format(k)
    dict_roll85[name] = season_roll
    dict_plot85[name] = season_forplot

In [ ]:
# Roll and sum
rtype_roll85 = {}

for k in rcp_monthly_dict85.keys():
    rtype_season = rcp_monthly_dict85[k].roll(month_2d=n_roll, roll_coords=True)
    rtype_sum = rtype_season.sum(dim='rgi_id')
    rtype_name = '{}'.format(k)
    rtype_roll85[rtype_name] = rtype_sum

# Output is a dictionary with summed values!

In [ ]:
rtype_roll85.keys()

In [ ]:
rtype_calc_s85 = {}
rtype_calc_e85 = {}

for k in rtype_roll85.keys():
    df_key = rtype_roll85[k]
    df_m_s = df_key.isel(time=slice(0, 10)).mean(dim='time').to_dataframe() * 1e-9
    df_m_s.columns = [c.replace('_monthly', '_m') for c in df_m_s.columns]
    df_m_s = df_m_s.clip(0)
    calc_name = '{}'.format(k)
    rtype_calc_s85[calc_name] = df_m_s

for k in rtype_roll85.keys():
    df_key = rtype_roll85[k]
    df_m_e = df_key.isel(time=slice(-11, -1)).mean(dim='time').to_dataframe() * 1e-9
    df_m_e.columns = [c.replace('_monthly', '_m') for c in df_m_e.columns]
    df_m_e = df_m_e.clip(0)
    calc_name = '{}'.format(k)
    rtype_calc_e85[calc_name] = df_m_e
    

In [ ]:
# IN USE: This is used for visualization of the 6-rolled hydro months, turns it into calendar months w/shift
season_roll2 = season_unroll85['NorESM1-M-rcp85-calc'].roll(month_2d=n_roll, roll_coords=True)

In [ ]:
# IN USE: This is used for plotting of months with values 1-12
index1 = season_unroll85['NorESM1-M-rcp85-calc']
index2 = index1['month_2d']

In [ ]:
# Make this into a visual
runoff_variables = ['melt_off_glacier_m', 'melt_on_glacier_m', 'liq_prcp_off_glacier_m', 'liq_prcp_on_glacier_m']
s_keys = rtype_calc_s85.keys()

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7), sharey=True);

for r in runoff_variables:
    runoff_s = rtype_calc_monthly(s_keys, rtype_calc_s85, r)
    runoff_e = rtype_calc_monthly(s_keys, rtype_calc_e85, r)
    xvals = index2
    ax1.plot(xvals, runoff_s['mean'], label=r)
    ax2.plot(xvals, runoff_e['mean'], label=r)
    
ax1.set(ylabel='Runoff [Mt]', title='2020-2030')
ax2.set(title='2090-2100')
# THIS WORKS
for ax in (ax1,ax2):
    ax.set(xticks = season_roll['month_2d'].values,
           xticklabels = season_roll2['month_2d'].values, #here, we want calendar year values
           xlabel = 'Calendar Month'
          );
    ax.legend();

plt.ylim([-0.5,22]);

## Visualize time slices!

In [ ]:
key_dict = {}

key_dict['rcp26'] = season_unroll26.keys()
key_dict['rcp45'] = season_unroll45.keys()
key_dict['rcp60'] = season_unroll60.keys()
key_dict['rcp85'] = season_unroll85.keys()

In [ ]:
# Let's try to get this to show each GCM as well as means!
time = [0, 20, 40, 80]

color_ts = ('#cce0f4', '#7bafe3', '#2b7ed1', '#2365a8')

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(9, 8), sharey=True, constrained_layout = True)

t_idx=0
for t in time:
    season_t = season_sliced(key_dict['rcp26'], dict_roll26, t)
    xvals = np.asarray(season_t.index)+1
    ax1.plot(xvals, season_t['mean'], label=t+2020, color=color_ts[t_idx])
    t_idx = t_idx + 1

t_idx=0
for t in time:
    season_t = season_sliced(key_dict['rcp45'], dict_roll45, t)
    xvals = np.asarray(season_t.index)+1
    ax2.plot(xvals, season_t['mean'], label=t+2020, color=color_ts[t_idx])
    t_idx = t_idx + 1

t_idx=0
for t in time:
    season_t = season_sliced(key_dict['rcp60'], dict_roll60, t)
    xvals = np.asarray(season_t.index)+1
    ax3.plot(xvals, season_t['mean'], label=t+2020, color=color_ts[t_idx])
    t_idx = t_idx + 1
    
t_idx=0
for t in time:
    season_t = season_sliced(key_dict['rcp85'], dict_roll85, t)
    xvals = np.asarray(season_t.index)+1
    ax4.plot(xvals, season_t['mean'], label=t+2020, color=color_ts[t_idx])
    t_idx = t_idx + 1

ax1.set(xticks = season_roll['month_2d'].values,
        xticklabels = season_roll2['month_2d'].values,
        title = 'RCP 2.6',
        ylabel = 'Runoff [Mt]');
ax2.set(xticks = season_roll['month_2d'].values,
        xticklabels = season_roll2['month_2d'].values,
        title = 'RCP 4.5');
ax3.set(xticks = season_roll['month_2d'].values,
        ylabel = 'Runoff [Mt]',
        xticklabels = season_roll2['month_2d'].values,
        title = 'RCP 6.0',
        xlabel = 'Month');
ax4.set(xticks = season_roll['month_2d'].values,
        xticklabels = season_roll2['month_2d'].values,
        title = 'RCP 8.5',
        xlabel = 'Month');
ax1.legend();

plt.suptitle('Multi-GCM mean seasonal runoff for four selected years', fontweight = 'bold', fontsize = 14);

This figure is wrong; the data is 7 months offset

## Compiled decadal runoff by RCP

I want to figure out how to add a legend to these plots.

In [ ]:
# Make this into a visual
runoff_variables = ['melt_off_glacier_m', 'melt_on_glacier_m']
sum_vars = ['liq_prcp_off_glacier_m', 'liq_prcp_on_glacier_m']

f, ((ax1, ax2), (ax3, ax4), (ax5, ax6), (ax7, ax8)) = plt.subplots(4, 2, figsize=(7, 10), sharey=True, constrained_layout=True)

color_pr = '#946cbd'
color_m = ('#d9985f', '#d16b6c')

s_keys = rtype_calc_s26.keys()
ct = 0
for r in runoff_variables:
    runoff_s = rtype_calc_monthly(s_keys, rtype_calc_s26, r)
    runoff_e = rtype_calc_monthly(s_keys, rtype_calc_e26, r)
    xvals = index2
    ax1.plot(xvals, runoff_s['mean'], label=r, color=color_m[ct])
    ax2.plot(xvals, runoff_e['mean'], label=r, color=color_m[ct])
    ct += 1
runoff_s = rtype_calc_monthly_2vars(s_keys, rtype_calc_s26, sum_vars[0], sum_vars[1])
runoff_e = rtype_calc_monthly_2vars(s_keys, rtype_calc_e26, sum_vars[0], sum_vars[1])
xvals = index2
ax1.plot(xvals, runoff_s['mean'], label=r, color=color_pr)
ax2.plot(xvals, runoff_e['mean'], label=r, color=color_pr)

s_keys = rtype_calc_s45.keys()
ct = 0
for r in runoff_variables:
    runoff_s = rtype_calc_monthly(s_keys, rtype_calc_s45, r)
    runoff_e = rtype_calc_monthly(s_keys, rtype_calc_e45, r)
    xvals = index2
    ax3.plot(xvals, runoff_s['mean'], label=r, color=color_m[ct])
    ax4.plot(xvals, runoff_e['mean'], label=r, color=color_m[ct])
    ct += 1
runoff_s = rtype_calc_monthly_2vars(s_keys, rtype_calc_s45, sum_vars[0], sum_vars[1])
runoff_e = rtype_calc_monthly_2vars(s_keys, rtype_calc_e45, sum_vars[0], sum_vars[1])
xvals = index2
ax3.plot(xvals, runoff_s['mean'], label=r, color=color_pr)
ax4.plot(xvals, runoff_e['mean'], label=r, color=color_pr)

s_keys = rtype_calc_s60.keys()
ct = 0
for r in runoff_variables:
    runoff_s = rtype_calc_monthly(s_keys, rtype_calc_s60, r)
    runoff_e = rtype_calc_monthly(s_keys, rtype_calc_e60, r)
    xvals = index2
    ax5.plot(xvals, runoff_s['mean'], label=r, color=color_m[ct])
    ax6.plot(xvals, runoff_e['mean'], label=r, color=color_m[ct])
    ct += 1
runoff_s = rtype_calc_monthly_2vars(s_keys, rtype_calc_s60, sum_vars[0], sum_vars[1])
runoff_e = rtype_calc_monthly_2vars(s_keys, rtype_calc_e60, sum_vars[0], sum_vars[1])
xvals = index2
ax5.plot(xvals, runoff_s['mean'], label=r, color=color_pr)
ax6.plot(xvals, runoff_e['mean'], label=r, color=color_pr)

s_keys = rtype_calc_s85.keys()
ct = 0
for r in runoff_variables:
    runoff_s = rtype_calc_monthly(s_keys, rtype_calc_s85, r)
    runoff_e = rtype_calc_monthly(s_keys, rtype_calc_e85, r)
    xvals = index2
    ax7.plot(xvals, runoff_s['mean'], label=r, color=color_m[ct])
    ax8.plot(xvals, runoff_e['mean'], label=r, color=color_m[ct])
    ct += 1
runoff_s = rtype_calc_monthly_2vars(s_keys, rtype_calc_s85, sum_vars[0], sum_vars[1])
runoff_e = rtype_calc_monthly_2vars(s_keys, rtype_calc_e85, sum_vars[0], sum_vars[1])
xvals = index2
ax7.plot(xvals, runoff_s['mean'], label=r, color=color_pr)
ax8.plot(xvals, runoff_e['mean'], label=r, color=color_pr)

ax1.set(ylabel='Runoff [Mt]', title='2020-2030') # this title and the next one are overwritten by my addition
ax2.set(title='2090-2100')
ax3.set(ylabel='Runoff [Mt]')
ax5.set(ylabel='Runoff [Mt]')
ax7.set(ylabel='Runoff [Mt]', xlabel='Month')
ax8.set(xlabel='Month')

# THIS WORKS
for ax in (ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8):
    ax.set(xticks=season_roll['month_2d'].values,
           xticklabels=season_roll2['month_2d'].values,  # here, we want calendar year values
           )

ax1.text(0.03, 0.9, 'RCP 2.6', transform=ax1.transAxes)
ax2.text(0.03, 0.9, 'RCP 2.6', transform=ax2.transAxes)
ax3.text(0.03, 0.9, 'RCP 4.5', transform=ax3.transAxes)
ax4.text(0.03, 0.9, 'RCP 4.5', transform=ax4.transAxes)
ax5.text(0.03, 0.9, 'RCP 6.0', transform=ax5.transAxes)
ax6.text(0.03, 0.9, 'RCP 6.0', transform=ax6.transAxes)
ax7.text(0.03, 0.9, 'RCP 8.5', transform=ax7.transAxes)
ax8.text(0.03, 0.9, 'RCP 8.5', transform=ax8.transAxes)

plt.suptitle('Decadal seasonal runoff averages by runoff component', fontweight='bold', fontsize=14)

plt.show()


In [ ]:
# this code is from ChatGPT

import matplotlib.pyplot as plt

# Define the runoff variables and colors
runoff_variables = ['melt_off_glacier_m', 'melt_on_glacier_m']
sum_vars = ['liq_prcp_off_glacier_m', 'liq_prcp_on_glacier_m']
color_pr = '#946cbd'
color_m = ['#d9985f', '#d16b6c']

# Create subplots with shared y-axis
fig, axes = plt.subplots(4, 2, figsize=(8, 10.5), sharey=True)
(ax1, ax2), (ax3, ax4), (ax5, ax6), (ax7, ax8) = axes

# Adjust the space between subplots
plt.subplots_adjust(wspace=0.05, hspace=0.2)

# Plotting function to avoid repetition
def plot_runoff(ax, rtype_calc_s, rtype_calc_e, s_keys, rtype_label):
    ct = 0
    for r in runoff_variables:
        runoff_s = rtype_calc_monthly(s_keys, rtype_calc_s, r)
        runoff_e = rtype_calc_monthly(s_keys, rtype_calc_e, r)
        xvals = index2
        ax[0].plot(xvals, runoff_s['mean'], label=r, color=color_m[ct])
        ax[1].plot(xvals, runoff_e['mean'], label=r, color=color_m[ct])
        ct += 1
    runoff_s = rtype_calc_monthly_2vars(s_keys, rtype_calc_s, sum_vars[0], sum_vars[1])
    runoff_e = rtype_calc_monthly_2vars(s_keys, rtype_calc_e, sum_vars[0], sum_vars[1])
    ax[0].plot(xvals, runoff_s['mean'], label='Total liquid precipitation', color=color_pr)
    ax[1].plot(xvals, runoff_e['mean'], label='Total liquid precipitation', color=color_pr)
    ax[0].set(ylabel='Runoff [Mt]')
    ax[0].text(0.03, 0.9, rtype_label, transform=ax[0].transAxes)
    ax[1].text(0.03, 0.9, rtype_label, transform=ax[1].transAxes)

# Plot each RCP scenario
plot_runoff([ax1, ax2], rtype_calc_s26, rtype_calc_e26, rtype_calc_s26.keys(), 'RCP 2.6')
plot_runoff([ax3, ax4], rtype_calc_s45, rtype_calc_e45, rtype_calc_s45.keys(), 'RCP 4.5')
plot_runoff([ax5, ax6], rtype_calc_s60, rtype_calc_e60, rtype_calc_s60.keys(), 'RCP 6.0')
plot_runoff([ax7, ax8], rtype_calc_s85, rtype_calc_e85, rtype_calc_s85.keys(), 'RCP 8.5')

# Set titles for the columns
ax1.set(title='2020-2030')
ax2.set(title='2090-2100')

# Set x-axis labels and ticks
for ax in (ax7, ax8):
    ax.set(xlabel='Month')
for ax in (ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8):
    ax.set(xticks=season_roll['month_2d'].values, xticklabels=season_roll2['month_2d'].values)

# Add a single legend to the right of the plots
handles, labels = ax1.get_legend_handles_labels()
fig.legend(handles, labels, loc='center right', bbox_to_anchor=(1.19, 0.84), borderaxespad=0.)

# Add a super title with adjusted vertical position
fig.suptitle('Decadal seasonal runoff averages by runoff component', fontweight='bold', fontsize=14, y=0.94)

plt.show()

### Plot of relative contribution on decade scale

In [ ]:
# Set up the dictionaries with values to use in the summing function
s_keys_dict = {}
s_keys_dict['rcp26'] = rtype_calc_s26.keys()
s_keys_dict['rcp45'] = rtype_calc_s45.keys()
s_keys_dict['rcp60'] = rtype_calc_s60.keys()
s_keys_dict['rcp85'] = rtype_calc_s85.keys()

rtype_calc_dict_s = {}
rtype_calc_dict_s['rcp26'] = rtype_calc_s26
rtype_calc_dict_s['rcp45'] = rtype_calc_s45
rtype_calc_dict_s['rcp60'] = rtype_calc_s60
rtype_calc_dict_s['rcp85'] = rtype_calc_s85

rtype_calc_dict_e = {}
rtype_calc_dict_e['rcp26'] = rtype_calc_e26
rtype_calc_dict_e['rcp45'] = rtype_calc_e45
rtype_calc_dict_e['rcp60'] = rtype_calc_e60
rtype_calc_dict_e['rcp85'] = rtype_calc_e85

In [ ]:
rcp = ('rcp26', 'rcp45', 'rcp60', 'rcp85')

all_rcp_dict_s = {}

for r in rcp:
    melt_on = rtype_calc_monthly(s_keys_dict[r], rtype_calc_dict_s[r], 'melt_on_glacier_m')
    melt_off = rtype_calc_monthly(s_keys_dict[r], rtype_calc_dict_s[r], 'melt_off_glacier_m')
    liq = rtype_calc_monthly_2vars(s_keys_dict[r], rtype_calc_dict_s[r], 'liq_prcp_off_glacier_m', 'liq_prcp_on_glacier_m')
    summed = melt_on + melt_off + liq
    m_on_name = 'm-on-{}'.format(r)
    m_off_name = 'm-off-{}'.format(r)
    liq_name = 'liq-{}'.format(r)
    sum_name = 'sum-{}'.format(r)
    all_rcp_dict_s[m_on_name] = melt_on
    all_rcp_dict_s[m_off_name] = melt_off
    all_rcp_dict_s[liq_name] = liq
    all_rcp_dict_s[sum_name] = summed

all_rcp_dict_e = {}

for r in rcp:
    melt_on = rtype_calc_monthly(s_keys_dict[r], rtype_calc_dict_e[r], 'melt_on_glacier_m')
    melt_off = rtype_calc_monthly(s_keys_dict[r], rtype_calc_dict_e[r], 'melt_off_glacier_m')
    liq = rtype_calc_monthly_2vars(s_keys_dict[r], rtype_calc_dict_e[r], 'liq_prcp_off_glacier_m', 'liq_prcp_on_glacier_m')
    summed = melt_on + melt_off + liq
    m_on_name = 'm-on-{}'.format(r)
    m_off_name = 'm-off-{}'.format(r)
    liq_name = 'liq-{}'.format(r)
    sum_name = 'sum-{}'.format(r)
    all_rcp_dict_e[m_on_name] = melt_on
    all_rcp_dict_e[m_off_name] = melt_off
    all_rcp_dict_e[liq_name] = liq
    all_rcp_dict_e[sum_name] = summed

In [ ]:
# Graph of relative importance of runoff for each part of the seasonal cycle (decade 1 and last)

f, ((ax1, ax2), (ax3, ax4), (ax5, ax6), (ax7, ax8)) = plt.subplots(4, 2, figsize=(9, 11), sharey=True, constrained_layout = True);

color_pr = '#946cbd'
color_m = ('#d9985f', '#d16b6c')

xvals = index2

# RCP 2.6
rcp_key = 'rcp26'
ax1.stackplot(xvals, all_rcp_dict_s['m-on-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_s['sum-{}'.format(rcp_key)]['mean'], 
              all_rcp_dict_s['m-off-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_s['sum-{}'.format(rcp_key)]['mean'],
              all_rcp_dict_s['liq-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_s['sum-{}'.format(rcp_key)]['mean'],
              labels=['Melt on glacier', 'Melt off glacier', 'Liquid precipitation'],
              colors=['#d16b6c', '#d9985f', '#946cbd']);

ax2.stackplot(xvals, all_rcp_dict_e['m-on-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_e['sum-{}'.format(rcp_key)]['mean'], 
              all_rcp_dict_e['m-off-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_e['sum-{}'.format(rcp_key)]['mean'],
              all_rcp_dict_e['liq-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_e['sum-{}'.format(rcp_key)]['mean'],
              labels=['Melt on glacier', 'Melt off glacier', 'Liquid precipitation'],
              colors=['#d16b6c', '#d9985f', '#946cbd']);

# RCP4.5
rcp_key = 'rcp45'
ax3.stackplot(xvals, all_rcp_dict_s['m-on-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_s['sum-{}'.format(rcp_key)]['mean'], 
              all_rcp_dict_s['m-off-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_s['sum-{}'.format(rcp_key)]['mean'],
              all_rcp_dict_s['liq-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_s['sum-{}'.format(rcp_key)]['mean'],
              labels=['Melt on glacier', 'Melt off glacier', 'Liquid precipitation'],
              colors=['#d16b6c', '#d9985f', '#946cbd']);

ax4.stackplot(xvals, all_rcp_dict_e['m-on-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_e['sum-{}'.format(rcp_key)]['mean'], 
              all_rcp_dict_e['m-off-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_e['sum-{}'.format(rcp_key)]['mean'],
              all_rcp_dict_e['liq-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_e['sum-{}'.format(rcp_key)]['mean'],
              labels=['Melt on glacier', 'Melt off glacier', 'Liquid precipitation'],
              colors=['#d16b6c', '#d9985f', '#946cbd']);

# RCP6.0
rcp_key = 'rcp60'
ax5.stackplot(xvals, all_rcp_dict_s['m-on-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_s['sum-{}'.format(rcp_key)]['mean'], 
              all_rcp_dict_s['m-off-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_s['sum-{}'.format(rcp_key)]['mean'],
              all_rcp_dict_s['liq-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_s['sum-{}'.format(rcp_key)]['mean'],
              labels=['Melt on glacier', 'Melt off glacier', 'Liquid precipitation'],
              colors=['#d16b6c', '#d9985f', '#946cbd']);

ax6.stackplot(xvals, all_rcp_dict_e['m-on-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_e['sum-{}'.format(rcp_key)]['mean'], 
              all_rcp_dict_e['m-off-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_e['sum-{}'.format(rcp_key)]['mean'],
              all_rcp_dict_e['liq-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_e['sum-{}'.format(rcp_key)]['mean'],
              labels=['Melt on glacier', 'Melt off glacier', 'Liquid precipitation'],
              colors=['#d16b6c', '#d9985f', '#946cbd']);

# RCP85
rcp_key = 'rcp85'
ax7.stackplot(xvals, all_rcp_dict_s['m-on-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_s['sum-{}'.format(rcp_key)]['mean'], 
              all_rcp_dict_s['m-off-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_s['sum-{}'.format(rcp_key)]['mean'],
              all_rcp_dict_s['liq-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_s['sum-{}'.format(rcp_key)]['mean'],
              labels=['Melt on glacier', 'Melt off glacier', 'Liquid precipitation'],
              colors=['#d16b6c', '#d9985f', '#946cbd']);

ax8.stackplot(xvals, all_rcp_dict_e['m-on-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_e['sum-{}'.format(rcp_key)]['mean'], 
              all_rcp_dict_e['m-off-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_e['sum-{}'.format(rcp_key)]['mean'],
              all_rcp_dict_e['liq-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_e['sum-{}'.format(rcp_key)]['mean'],
              labels=['Melt on glacier', 'Melt off glacier', 'Liquid precipitation'],
              colors=['#d16b6c', '#d9985f', '#946cbd']);

#ax1.legend(bbox_to_anchor=(0.44, 0.95));

plt.suptitle('Relative seasonal contribution of runoff components', fontweight = 'bold', fontsize = 14);

ax1.set(ylabel='% Total runoff', title='2020-2030')
ax2.set(title='2090-2100')
ax3.set(ylabel='% Total runoff')
ax5.set(ylabel='% Total runoff')
ax7.set(ylabel='% Total runoff', xlabel = 'Month')
ax8.set(xlabel = 'Month')
# THIS WORKS
for ax in (ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8):
    ax.set(xticks = season_roll['month_2d'].values,
           xticklabels = season_roll2['month_2d'].values #here, we want calendar year values
          );
# Add RCP labels as titles (from chatgpt)
ax1.set_title('RCP 2.6: 2020-2030', fontsize=10)
ax2.set_title('RCP 2.6: 2090-2100', fontsize=10)
ax3.set_title('RCP 4.5: 2020-2030', fontsize=10)
ax4.set_title('RCP 4.5: 2090-2100', fontsize=10)
ax5.set_title('RCP 6.0: 2020-2030', fontsize=10)
ax6.set_title('RCP 6.0: 2090-2100', fontsize=10)
ax7.set_title('RCP 8.5: 2020-2030', fontsize=10)
ax8.set_title('RCP 8.5: 2090-2100', fontsize=10)

#plt.savefig('Figures/Seasonal/Decade_av/4x2_relative_importance_newMay4.png', bbox_inches='tight')

# Step 2: try with another GCM

In [ ]:
# This is place to add GCMs and dates; it only needs to be in one place, so I commented it out

# # Define variables that will be used for each test
# base = '/Users/ziggygoddard/Documents/summer24/'

# gcm_list = ['NorESM1-M', 'CCSM4'] # just the current gcm

# date_list = ['2024-06-05', '2024-06-06'] # export date (date I downloaded the data)

# rcp_list = ['rcp26', 'rcp45', 'rcp60', 'rcp85'] # this stays the same

# gcm_w60 = ['NorESM1-M', 'CCSM4'] 

# all_monthly_dict = {}

# # These have to be files that I have downloaded

## Try to make a mega-dictionary with all RCPs

In [ ]:
# create dictionary
all_monthly_dict = {}

In [ ]:
rcp_monthly_dict = {}

for rcp in rcp_list:
    rcp_dict = {}
    for i, gcm in enumerate(gcm_list):
        date = date_list[i]  # Ensure the date corresponds to the correct GCM
        filepath = f"{base}{date}-gcm_data_{gcm}_{rcp}_compiled_output.nc"
        print(f"Trying to open file: {filepath}")
        if rcp != 'rcp60':
            rcp_value = open_series(fpath_stem=base, which_date=date, which_gcm=gcm, which_rcp=rcp)
            k = '{}-{}'.format(gcm, rcp)
            rcp_dict[k] = rcp_value
        else:
            if gcm in gcm_w60:
                rcp_value = open_series(fpath_stem=base, which_date=date, which_gcm=gcm, which_rcp=rcp)
                k = '{}-{}'.format(gcm, rcp)
                rcp_dict[k] = rcp_value
    rcp_monthly_dict[rcp] = rcp_dict

In [ ]:
x=0
rcp = ['rcp26', 'rcp45', 'rcp60', 'rcp85']

for r in rcp:
    x = 0
    for gcm in gcm_list:
        if r != 'rcp60':
            rcp_value = open_series(fpath_stem=base, which_date=date_list[x], which_gcm=gcm, which_rcp=r)
            k = '{}-{}'.format(gcm, r)
            all_monthly_dict[k] = rcp_value
        else:
            if gcm in gcm_w60:
                rcp_value = open_series(fpath_stem=base, which_date=date_list[x], which_gcm=gcm, which_rcp=r)
                k = '{}-{}'.format(gcm, r)
                all_monthly_dict[k]=rcp_value
        x += 1

In [ ]:
# Define keys
keys_all = all_monthly_dict.keys()

In [ ]:
all_unrolled = {}
for k in keys_all:
    calculation = calc_seasonal(all_monthly_dict[k])
    z = '{}'.format(k)
    all_unrolled[z] = calculation

Roll values for plotting

In [ ]:
n_roll = 3

dict_roll_all = {}
dict_plot_all = {}

for k in keys_all:
    season_roll = all_unrolled[k].roll(month_2d=n_roll, roll_coords=True)
    season_forplot = all_unrolled[k].roll(month_2d=n_roll, roll_coords=False)
    name = '{}'.format(k)
    dict_roll_all[name] = season_roll
    dict_plot_all[name] = season_forplot

In [ ]:
dict_roll_all

Take sum over rgi id

In [ ]:
rtype_all = {}

for k in keys_all:
    rtype_season = all_monthly_dict[k].roll(roll_coords=True)
    rtype_sum = rtype_season.sum(dim='rgi_id')
    rtype_name = '{}'.format(k)
    rtype_all[rtype_name] = rtype_sum

In [ ]:
# Make a dictionary with all data for all GCMs for each RCP
rcp_monthly_dict = {}

for rcp in rcp_list:
    rcp_dict = {}
    for gcm in gcm_list:
        for date in date_list:
            # Check if the RCP is not 'rcp60' or the GCM is in gcm_w60
            if rcp != 'rcp60' or gcm in gcm_w60:
                filepath = base + '{}-gcm_data_{}_{}_compiled_output.nc'.format(date, gcm, rcp)
                # Check if the file exists before attempting to open it
                if os.path.exists(filepath):
                    rcp_value = open_series(fpath_stem=base, which_date=date, which_gcm=gcm, which_rcp=rcp)
                    k = '{}-{}'.format(gcm, rcp)
                    rcp_dict[k] = rcp_value
    rcp_monthly_dict[rcp] = rcp_dict

### Calculate rcp26

In [ ]:
# Make a dictionary with all data for all GCMs (this will only output one rcp!)
rcp_monthly_dict26 = {}

x=0
rcp = 'rcp26'

# Per rcp:
for gcm in gcm_list:
    if rcp != 'rcp60':
        rcp_value = open_series(fpath_stem=base, which_date=date_list[x], which_gcm=gcm, which_rcp=rcp)
        k = '{}-{}'.format(gcm, rcp)
        rcp_monthly_dict26[k]=rcp_value
    else:
        if gcm in gcm_w60:
            rcp_value = open_series(fpath_stem=base, which_date=date_list[x], which_gcm=gcm, which_rcp=rcp)
            k = '{}-{}'.format(gcm, rcp)
            rcp_monthly_dict26[k]=rcp_value
    x += 1

In [ ]:
rcp_monthly_dict26['CCSM4-rcp26']

In [ ]:
# Bring in your dictionary of unrolled values to find TOTAL RUNOFF (for time slice plot)
season_unroll26 = {}
for k in rcp_monthly_dict26.keys():
    calculation = calc_seasonal(rcp_monthly_dict26[k])
    z = '{}-calc'.format(k)
    season_unroll26[z] = calculation

In [ ]:
# make current gcm

season_unroll26['CCSM4-rcp26-calc']

In [ ]:
# define keys to use in next step
keys26 = season_unroll26.keys()

In [ ]:
# Roll coordinates!
dict_roll26 = {}
dict_plot26 = {}

for k in keys26:
    season_roll = season_unroll26[k].roll(month_2d=n_roll, roll_coords=False)
    season_forplot = season_unroll26[k].roll(month_2d=n_roll, roll_coords=False) 
    name = '{}'.format(k)
    dict_roll26[name] = season_roll
    dict_plot26[name] = season_forplot

In [ ]:
# Roll and sum
rtype_roll26 = {}

for k in rcp_monthly_dict26.keys():
    rtype_season = rcp_monthly_dict26[k].roll(month_2d=n_roll, roll_coords=True)
    rtype_sum = rtype_season.sum(dim='rgi_id')
    rtype_name = '{}'.format(k)
    rtype_roll26[rtype_name] = rtype_sum

# Output is a dictionary with summed values!

In [ ]:
rtype_roll26.keys()

In [ ]:
rtype_calc_s26 = {}
rtype_calc_e26 = {}

for k in rtype_roll26.keys():
    df_key = rtype_roll26[k]
    df_m_s = df_key.isel(time=slice(0, 10)).mean(dim='time').to_dataframe() * 1e-9
    df_m_s.columns = [c.replace('_monthly', '_m') for c in df_m_s.columns]
    df_m_s = df_m_s.clip(0)
    calc_name = '{}'.format(k)
    rtype_calc_s26[calc_name] = df_m_s

for k in rtype_roll26.keys():
    df_key = rtype_roll26[k]
    df_m_e = df_key.isel(time=slice(-11, -1)).mean(dim='time').to_dataframe() * 1e-9
    df_m_e.columns = [c.replace('_monthly', '_m') for c in df_m_e.columns]
    df_m_e = df_m_e.clip(0)
    calc_name = '{}'.format(k)
    rtype_calc_e26[calc_name] = df_m_e
    

In [ ]:
season_roll2 = season_unroll26['CCSM4-rcp26-calc'].roll(month_2d=n_roll, roll_coords=True)

In [ ]:
# IN USE: This is used for plotting of months with values 1-12
index1 = season_unroll26['CCSM4-rcp26-calc']
index2 = index1['month_2d']

In [ ]:
# Make this into a visual
runoff_variables = ['melt_off_glacier_m', 'melt_on_glacier_m', 'liq_prcp_off_glacier_m', 'liq_prcp_on_glacier_m']
s_keys = rtype_calc_s26.keys()

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7), sharey=True);

for r in runoff_variables:
    runoff_s = rtype_calc_monthly(s_keys, rtype_calc_s26, r)
    runoff_e = rtype_calc_monthly(s_keys, rtype_calc_e26, r)
    xvals = index2
    ax1.plot(xvals, runoff_s['mean'], label=r)
    ax2.plot(xvals, runoff_e['mean'], label=r)

ax1.set(ylabel='Runoff [Mt]', title='2020-2030')
ax2.set(title='2090-2100')
# THIS WORKS
for ax in (ax1,ax2):
    ax.set(xticks = season_roll['month_2d'].values,
           xticklabels = season_roll2['month_2d'].values, #here, we want calendar year values
           xlabel = 'Calendar Month'
          );
    ax.legend();

Aside from the temporal offset, these graphs are different than Molly's even though the GCM is the same. This is an issue.
resolved

### Calculate rcp45

In [ ]:
# Make a dictionary with all data for all GCMs (this will only output one rcp!)
rcp_monthly_dict45 = {}

x=0
rcp = 'rcp45'

# Per rcp:
for gcm in gcm_list:
    if rcp != 'rcp60':
        rcp_value = open_series(fpath_stem=base, which_date=date_list[x], which_gcm=gcm, which_rcp=rcp)
        k = '{}-{}'.format(gcm, rcp)
        rcp_monthly_dict45[k]=rcp_value
    else:
        if gcm in gcm_w60:
            rcp_value = open_series(fpath_stem=base, which_date=date_list[x], which_gcm=gcm, which_rcp=rcp)
            k = '{}-{}'.format(gcm, rcp)
            rcp_monthly_dict45[k]=rcp_value
    x += 1

In [ ]:
# Bring in your dictionary of unrolled values
season_unroll45 = {}
for k in rcp_monthly_dict45.keys():
    calculation = calc_seasonal(rcp_monthly_dict45[k])
    z = '{}-calc'.format(k)
    season_unroll45[z] = calculation

In [ ]:
# define keys to use in next step
keys45 = season_unroll45.keys()

In [ ]:
# Roll coordinates!
dict_roll45 = {}
dict_plot45 = {}

for k in keys45:
    season_roll = season_unroll45[k].roll(month_2d=n_roll, roll_coords=False)
    season_forplot = season_unroll45[k].roll(month_2d=n_roll, roll_coords=False)
    name = '{}'.format(k)
    dict_roll45[name] = season_roll
    dict_plot45[name] = season_forplot

In [ ]:
# Roll and sum
rtype_roll45 = {}

for k in rcp_monthly_dict45.keys():
    rtype_season = rcp_monthly_dict45[k].roll(month_2d=n_roll, roll_coords=True)
    rtype_sum = rtype_season.sum(dim='rgi_id')
    rtype_name = '{}'.format(k)
    rtype_roll45[rtype_name] = rtype_sum

# Output is a dictionary with summed values!

In [ ]:
rtype_roll45.keys()

In [ ]:
rtype_calc_s45 = {}
rtype_calc_e45 = {}

for k in rtype_roll45.keys():
    df_key = rtype_roll45[k]
    df_m_s = df_key.isel(time=slice(0, 10)).mean(dim='time').to_dataframe() * 1e-9
    df_m_s.columns = [c.replace('_monthly', '_m') for c in df_m_s.columns]
    df_m_s = df_m_s.clip(0)
    calc_name = '{}'.format(k)
    rtype_calc_s45[calc_name] = df_m_s

for k in rtype_roll45.keys():
    df_key = rtype_roll45[k]
    df_m_e = df_key.isel(time=slice(-11, -1)).mean(dim='time').to_dataframe() * 1e-9
    df_m_e.columns = [c.replace('_monthly', '_m') for c in df_m_e.columns]
    df_m_e = df_m_e.clip(0)
    calc_name = '{}'.format(k)
    rtype_calc_e45[calc_name] = df_m_e
    

In [ ]:
season_roll2 = season_unroll45['CCSM4-rcp45-calc'].roll(month_2d=n_roll, roll_coords=True)

In [ ]:
# IN USE: This is used for plotting of months with values 1-12
index1 = season_unroll45['CCSM4-rcp45-calc']
index2 = index1['month_2d']

In [ ]:
# Make this into a visual
runoff_variables = ['melt_off_glacier_m', 'melt_on_glacier_m', 'liq_prcp_off_glacier_m', 'liq_prcp_on_glacier_m']
s_keys = rtype_calc_s45.keys()

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7), sharey=True);

for r in runoff_variables:
    runoff_s = rtype_calc_monthly(s_keys, rtype_calc_s45, r)
    runoff_e = rtype_calc_monthly(s_keys, rtype_calc_e45, r)
    xvals = index2
    ax1.plot(xvals, runoff_s['mean'], label=r)
    ax2.plot(xvals, runoff_e['mean'], label=r)
    
ax1.set(ylabel='Runoff [Mt]', title='2020-2030')
ax2.set(title='2090-2100')
# THIS WORKS
for ax in (ax1,ax2):
    ax.set(xticks = season_roll['month_2d'].values,
           xticklabels = season_roll2['month_2d'].values, #here, we want calendar year values
           xlabel = 'Calendar Month'
          );
    ax.legend();

plt.ylim([-0.5,20]);

These plots look different than Molly's (later peak runoff)

### Calculate rcp60

In [ ]:
# Make a dictionary with all data for all GCMs (this will only output one rcp!)
rcp_monthly_dict60 = {}

x=0
rcp = 'rcp60'

# Per rcp:
for gcm in gcm_list:
    if rcp != 'rcp60':
        rcp_value = open_series(fpath_stem=base, which_date=date_list[x], which_gcm=gcm, which_rcp=rcp)
        k = '{}-{}'.format(gcm, rcp)
        rcp_monthly_dict60[k]=rcp_value
    else:
        if gcm in gcm_w60:
            rcp_value = open_series(fpath_stem=base, which_date=date_list[x], which_gcm=gcm, which_rcp=rcp)
            k = '{}-{}'.format(gcm, rcp)
            rcp_monthly_dict60[k]=rcp_value
    x += 1

In [ ]:
# Bring in your dictionary of unrolled values
season_unroll60 = {}
for k in rcp_monthly_dict60.keys():
    calculation = calc_seasonal(rcp_monthly_dict60[k])
    z = '{}-calc'.format(k)
    season_unroll60[z] = calculation

In [ ]:
# define keys to use in next step
keys60 = season_unroll60.keys()

In [ ]:
# Roll coordinates!
dict_roll60 = {}
dict_plot60 = {}

for k in keys60:
    season_roll = season_unroll60[k].roll(month_2d=n_roll, roll_coords=False)
    season_forplot = season_unroll60[k].roll(month_2d=n_roll, roll_coords=False)
    name = '{}'.format(k)
    dict_roll60[name] = season_roll
    dict_plot60[name] = season_forplot

In [ ]:
# Roll and sum
rtype_roll60 = {}

for k in rcp_monthly_dict60.keys():
    rtype_season = rcp_monthly_dict60[k].roll(month_2d=n_roll, roll_coords=True)
    rtype_sum = rtype_season.sum(dim='rgi_id')
    rtype_name = '{}'.format(k)
    rtype_roll60[rtype_name] = rtype_sum

# Output is a dictionary with summed values!

In [ ]:
rtype_roll60.keys()

In [ ]:
rtype_calc_s60 = {}
rtype_calc_e60 = {}

for k in rtype_roll60.keys():
    df_key = rtype_roll60[k]
    df_m_s = df_key.isel(time=slice(0, 10)).mean(dim='time').to_dataframe() * 1e-9
    df_m_s.columns = [c.replace('_monthly', '_m') for c in df_m_s.columns]
    df_m_s = df_m_s.clip(0)
    calc_name = '{}'.format(k)
    rtype_calc_s60[calc_name] = df_m_s

for k in rtype_roll60.keys():
    df_key = rtype_roll60[k]
    df_m_e = df_key.isel(time=slice(-11, -1)).mean(dim='time').to_dataframe() * 1e-9
    df_m_e.columns = [c.replace('_monthly', '_m') for c in df_m_e.columns]
    df_m_e = df_m_e.clip(0)
    calc_name = '{}'.format(k)
    rtype_calc_e60[calc_name] = df_m_e
    

In [ ]:
# IN USE: This is used for visualization of the 6-rolled hydro months, turns it into calendar months w/shift
season_roll2 = season_unroll60['CCSM4-rcp60-calc'].roll(month_2d=n_roll, roll_coords=True)

In [ ]:
# IN USE: This is used for plotting of months with values 1-12
index1 = season_unroll60['CCSM4-rcp60-calc']
index2 = index1['month_2d']

In [ ]:
# Make this into a visual
runoff_variables = ['melt_off_glacier_m', 'melt_on_glacier_m', 'liq_prcp_off_glacier_m', 'liq_prcp_on_glacier_m']
s_keys = rtype_calc_s60.keys()

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7), sharey=True);

for r in runoff_variables:
    runoff_s = rtype_calc_monthly(s_keys, rtype_calc_s60, r)
    runoff_e = rtype_calc_monthly(s_keys, rtype_calc_e60, r)
    xvals = index2
    ax1.plot(xvals, runoff_s['mean'], label=r)
    ax2.plot(xvals, runoff_e['mean'], label=r)
    
ax1.set(ylabel='Runoff [Mt]', title='2020-2030')
ax2.set(title='2090-2100')
# THIS WORKS
for ax in (ax1,ax2):
    ax.set(xticks = season_roll['month_2d'].values,
           xticklabels = season_roll2['month_2d'].values, #here, we want calendar year values
           xlabel = 'Calendar Month'
          );
    ax.legend();

plt.ylim([-0.5,22]);

### Calculate rcp85

In [ ]:
# Make a dictionary with all data for all GCMs (this will only output one rcp!)
rcp_monthly_dict85 = {}

x=0
rcp = 'rcp85'

# Per rcp:
for gcm in gcm_list:
    if rcp != 'rcp60':
        rcp_value = open_series(fpath_stem=base, which_date=date_list[x], which_gcm=gcm, which_rcp=rcp)
        k = '{}-{}'.format(gcm, rcp)
        rcp_monthly_dict85[k]=rcp_value
    else:
        if gcm in gcm_w60:
            rcp_value = open_series(fpath_stem=base, which_date=date_list[x], which_gcm=gcm, which_rcp=rcp)
            k = '{}-{}'.format(gcm, rcp)
            rcp_monthly_dict85[k]=rcp_value
    x += 1

In [ ]:
# Bring in your dictionary of unrolled values
season_unroll85 = {}
for k in rcp_monthly_dict85.keys():
    calculation = calc_seasonal(rcp_monthly_dict85[k])
    z = '{}-calc'.format(k)
    season_unroll85[z] = calculation

In [ ]:
# define keys to use in next step
keys85 = season_unroll85.keys()

In [ ]:
# Roll coordinates!
dict_roll85 = {}
dict_plot85 = {}

for k in keys85:
    season_roll = season_unroll85[k].roll(month_2d=n_roll, roll_coords=False)
    season_forplot = season_unroll85[k].roll(month_2d=n_roll, roll_coords=False)
    name = '{}'.format(k)
    dict_roll85[name] = season_roll
    dict_plot85[name] = season_forplot

In [ ]:
# Roll and sum
rtype_roll85 = {}

for k in rcp_monthly_dict85.keys():
    rtype_season = rcp_monthly_dict85[k].roll(month_2d=n_roll, roll_coords=True)
    rtype_sum = rtype_season.sum(dim='rgi_id')
    rtype_name = '{}'.format(k)
    rtype_roll85[rtype_name] = rtype_sum

# Output is a dictionary with summed values!

In [ ]:
rtype_roll85.keys()

In [ ]:
rtype_calc_s85 = {}
rtype_calc_e85 = {}

for k in rtype_roll85.keys():
    df_key = rtype_roll85[k]
    df_m_s = df_key.isel(time=slice(0, 10)).mean(dim='time').to_dataframe() * 1e-9
    df_m_s.columns = [c.replace('_monthly', '_m') for c in df_m_s.columns]
    df_m_s = df_m_s.clip(0)
    calc_name = '{}'.format(k)
    rtype_calc_s85[calc_name] = df_m_s

for k in rtype_roll85.keys():
    df_key = rtype_roll85[k]
    df_m_e = df_key.isel(time=slice(-11, -1)).mean(dim='time').to_dataframe() * 1e-9
    df_m_e.columns = [c.replace('_monthly', '_m') for c in df_m_e.columns]
    df_m_e = df_m_e.clip(0)
    calc_name = '{}'.format(k)
    rtype_calc_e85[calc_name] = df_m_e
    

In [ ]:
season_roll2 = season_unroll85['CCSM4-rcp85-calc'].roll(month_2d=n_roll, roll_coords=True)

In [ ]:
# IN USE: This is used for plotting of months with values 1-12
index1 = season_unroll85['CCSM4-rcp85-calc']
index2 = index1['month_2d']

In [ ]:
# Make this into a visual
runoff_variables = ['melt_off_glacier_m', 'melt_on_glacier_m', 'liq_prcp_off_glacier_m', 'liq_prcp_on_glacier_m']
s_keys = rtype_calc_s85.keys()

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7), sharey=True);

for r in runoff_variables:
    runoff_s = rtype_calc_monthly(s_keys, rtype_calc_s85, r)
    runoff_e = rtype_calc_monthly(s_keys, rtype_calc_e85, r)
    xvals = index2
    ax1.plot(xvals, runoff_s['mean'], label=r)
    ax2.plot(xvals, runoff_e['mean'], label=r)
    
ax1.set(ylabel='Runoff [Mt]', title='2020-2030')
ax2.set(title='2090-2100')
# THIS WORKS
for ax in (ax1,ax2):
    ax.set(xticks = season_roll['month_2d'].values,
           xticklabels = season_roll2['month_2d'].values, #here, we want calendar year values
           xlabel = 'Calendar Month'
          );
    ax.legend();

plt.ylim([-0.5,22]);

# Step 3: multi-GCM visualizations

## Visualize time slices!

In [ ]:
key_dict = {}

key_dict['rcp26'] = season_unroll26.keys()
key_dict['rcp45'] = season_unroll45.keys()
key_dict['rcp60'] = season_unroll60.keys()
key_dict['rcp85'] = season_unroll85.keys()

In [ ]:
# Let's try to get this to show each GCM as well as means!
time = [0, 20, 40, 80]

color_ts = ('#cce0f4', '#7bafe3', '#2b7ed1', '#2365a8')

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(9, 8), sharey=True, constrained_layout = True)       

t_idx=0
for t in time:
    season_t = season_sliced(key_dict['rcp26'], dict_roll26, t)
    xvals = np.asarray(season_t.index)+1
    ax1.plot(xvals, season_t['mean'], label=t+2020, color=color_ts[t_idx])
    t_idx = t_idx + 1

t_idx=0
for t in time:
    season_t = season_sliced(key_dict['rcp45'], dict_roll45, t)
    xvals = np.asarray(season_t.index)+1
    ax2.plot(xvals, season_t['mean'], label=t+2020, color=color_ts[t_idx])
    t_idx = t_idx + 1

t_idx=0
for t in time:
    season_t = season_sliced(key_dict['rcp60'], dict_roll60, t)
    xvals = np.asarray(season_t.index)+1
    ax3.plot(xvals, season_t['mean'], label=t+2020, color=color_ts[t_idx])
    t_idx = t_idx + 1
    
t_idx=0
for t in time:
    season_t = season_sliced(key_dict['rcp85'], dict_roll85, t)
    xvals = np.asarray(season_t.index)+1
    ax4.plot(xvals, season_t['mean'], label=t+2020, color=color_ts[t_idx])
    t_idx = t_idx + 1

ax1.set(xticks = season_roll['month_2d'].values,
        xticklabels = season_roll2['month_2d'].values,
        title = 'RCP 2.6',
        ylabel = 'Runoff [Mt]');
ax2.set(xticks = season_roll['month_2d'].values,
        xticklabels = season_roll2['month_2d'].values,
        title = 'RCP 4.5');
ax3.set(xticks = season_roll['month_2d'].values,
        ylabel = 'Runoff [Mt]',
        xticklabels = season_roll2['month_2d'].values,
        title = 'RCP 6.0',
        xlabel = 'Month');
ax4.set(xticks = season_roll['month_2d'].values,
        xticklabels = season_roll2['month_2d'].values,
        title = 'RCP 8.5',
        xlabel = 'Month');
ax1.legend();

plt.suptitle('Multi-GCM mean seasonal runoff for four selected years', fontweight = 'bold', fontsize = 14);

In [ ]:
time = [0, 20, 40, 80]

from matplotlib import cm
#color_ts = ('#c2f2f2', '#81cfdb', '#4c858f', '#23393d')
#New colors 5/4/23
color_ts = ('#cce0f4', '#7bafe3', '#2b7ed1', '#2365a8')

fig, ((ax1, ax2, ax3, ax4), (ax5, ax6, ax7, ax8), (ax9, ax10, ax11, ax12), (ax13, ax14, ax15, ax16)) = plt.subplots(4, 4, figsize=(10, 9), sharex=True, sharey=True, constrained_layout = True)

# Make first row of RCPs for time t=0
t = time[0]
for k in season_unroll26.keys():
    color_idx = 0
    season_t = season_sliced(key_dict['rcp26'], dict_roll26, t)
    xvals = np.asarray(season_t.index)+1
    ax1.plot(xvals, season_t, linewidth=0.15, color=color_ts[color_idx], alpha=0.8)
ax1.plot(xvals, season_t['mean'], color='k', linewidth=1.5)

for k in season_unroll45.keys():
    color_idx = 0
    season_t = season_sliced(key_dict['rcp45'], dict_roll45, t)
    xvals = np.asarray(season_t.index)+1
    ax2.plot(xvals, season_t, linewidth=0.15, color=color_ts[color_idx], alpha=0.8)
ax2.plot(xvals, season_t['mean'], color='k', linewidth=1.5)

for k in season_unroll60.keys():
    color_idx = 0
    season_t = season_sliced(key_dict['rcp60'], dict_roll60, t)
    xvals = np.asarray(season_t.index)+1
    ax3.plot(xvals, season_t, linewidth=0.15, color=color_ts[color_idx], alpha=0.8)
ax3.plot(xvals, season_t['mean'], color='k', linewidth=1.5)

for k in season_unroll85.keys():
    color_idx = 0
    season_t = season_sliced(key_dict['rcp85'], dict_roll85, t)
    xvals = np.asarray(season_t.index)+1
    ax4.plot(xvals, season_t, linewidth=0.15, color=color_ts[color_idx], alpha=0.8)
ax4.plot(xvals, season_t['mean'], color='k', linewidth=1.5)


# Make second row of RCPs for time t=20
t = time[1]
for k in season_unroll26.keys():
    color_idx = 1
    season_t = season_sliced(key_dict['rcp26'], dict_roll26, t)
    xvals = np.asarray(season_t.index)+1
    ax5.plot(xvals, season_t, linewidth=0.15, color=color_ts[color_idx], alpha=0.7)
ax5.plot(xvals, season_t['mean'], color='k', linewidth=1.5)

for k in season_unroll45.keys():
    color_idx = 1
    season_t = season_sliced(key_dict['rcp45'], dict_roll45, t)
    xvals = np.asarray(season_t.index)+1
    ax6.plot(xvals, season_t, linewidth=0.15, color=color_ts[color_idx], alpha=0.7)
ax6.plot(xvals, season_t['mean'], color='k', linewidth=1.5)

for k in season_unroll60.keys():
    color_idx = 1
    season_t = season_sliced(key_dict['rcp60'], dict_roll60, t)
    xvals = np.asarray(season_t.index)+1
    ax7.plot(xvals, season_t, linewidth=0.15, color=color_ts[color_idx], alpha=0.7)
ax7.plot(xvals, season_t['mean'], color='k', linewidth=1.5)

for k in season_unroll85.keys():
    color_idx = 1
    season_t = season_sliced(key_dict['rcp85'], dict_roll85, t)
    xvals = np.asarray(season_t.index)+1
    ax8.plot(xvals, season_t, linewidth=0.15, color=color_ts[color_idx], alpha=0.7)
ax8.plot(xvals, season_t['mean'], color='k', linewidth=1.5)


# Make third row of RCPs for time t=40
t = time[2]
for k in season_unroll26.keys():
    color_idx = 2
    season_t = season_sliced(key_dict['rcp26'], dict_roll26, t)
    xvals = np.asarray(season_t.index)+1
    ax9.plot(xvals, season_t, linewidth=0.15, color=color_ts[color_idx], alpha=0.7)
ax9.plot(xvals, season_t['mean'], color='k', linewidth=1.5)

for k in season_unroll45.keys():
    color_idx = 2
    season_t = season_sliced(key_dict['rcp45'], dict_roll45, t)
    xvals = np.asarray(season_t.index)+1
    ax10.plot(xvals, season_t, linewidth=0.15, color=color_ts[color_idx], alpha=0.7)
ax10.plot(xvals, season_t['mean'], color='k', linewidth=1.5)

for k in season_unroll60.keys():
    color_idx = 2
    season_t = season_sliced(key_dict['rcp60'], dict_roll60, t)
    xvals = np.asarray(season_t.index)+1
    ax11.plot(xvals, season_t, linewidth=0.15, color=color_ts[color_idx], alpha=0.7)
ax11.plot(xvals, season_t['mean'], color='k', linewidth=1.5)

for k in season_unroll85.keys():
    color_idx = 2
    season_t = season_sliced(key_dict['rcp85'], dict_roll85, t)
    xvals = np.asarray(season_t.index)+1
    ax12.plot(xvals, season_t, linewidth=0.15, color=color_ts[color_idx], alpha=0.7)
ax12.plot(xvals, season_t['mean'], color='k', linewidth=1.5)


# Make fourth row of RCPs for time t=80
t = time[3]
for k in season_unroll26.keys():
    color_idx = 3
    season_t = season_sliced(key_dict['rcp26'], dict_roll26, t)
    xvals = np.asarray(season_t.index)+1
    ax13.plot(xvals, season_t, linewidth=0.15, color=color_ts[color_idx], alpha=0.7)
ax13.plot(xvals, season_t['mean'], color='k', linewidth=1.5)

for k in season_unroll45.keys():
    color_idx = 3
    season_t = season_sliced(key_dict['rcp45'], dict_roll45, t)
    xvals = np.asarray(season_t.index)+1
    ax14.plot(xvals, season_t, linewidth=0.15, color=color_ts[color_idx], alpha=0.7)
ax14.plot(xvals, season_t['mean'], color='k', linewidth=1.5)

for k in season_unroll60.keys():
    color_idx = 3
    season_t = season_sliced(key_dict['rcp60'], dict_roll60, t)
    xvals = np.asarray(season_t.index)+1
    ax15.plot(xvals, season_t, linewidth=0.15, color=color_ts[color_idx], alpha=0.7)
ax15.plot(xvals, season_t['mean'], color='k', linewidth=1.5)

for k in season_unroll85.keys():
    color_idx = 3
    season_t = season_sliced(key_dict['rcp85'], dict_roll85, t)
    xvals = np.asarray(season_t.index)+1
    ax16.plot(xvals, season_t, linewidth=0.15, color=color_ts[color_idx], alpha=0.7)
ax16.plot(xvals, season_t['mean'], color='k', linewidth=1.5)
    
ax1.set(xticks = season_roll['month_2d'].values,
       xticklabels = season_roll2['month_2d'].values,
       title = 'RCP 2.6');
ax2.set(xticks = season_roll['month_2d'].values,
       xticklabels = season_roll2['month_2d'].values,
       title = 'RCP 4.5');
ax3.set(xticks = season_roll['month_2d'].values,
       xticklabels = season_roll2['month_2d'].values,
       title = 'RCP 6.0');
ax4.set(xticks = season_roll['month_2d'].values,
       xticklabels = season_roll2['month_2d'].values,
       title = 'RCP 8.5');

ax1.set(ylabel = 'Runoff [Mt]')
ax5.set(ylabel = 'Runoff [Mt]')
ax9.set(ylabel = 'Runoff [Mt]')
ax13.set(ylabel = 'Runoff [Mt]',
        xlabel = 'Month')
ax14.set(xlabel = 'Month')
ax15.set(xlabel = 'Month')
ax16.set(xlabel = 'Month')

ax13.tick_params(axis='x', which='major', labelsize=8)
ax14.tick_params(axis='x', which='major', labelsize=8)
ax15.tick_params(axis='x', which='major', labelsize=8)
ax16.tick_params(axis='x', which='major', labelsize=8)

plt.suptitle('Multi-GCM seasonal runoff for four selected years', fontweight = 'bold', fontsize = 14);

#plt.savefig('Figures/Seasonal/Slice/all_rcps_all_GCMs_16mega_may4', bbox_inches='tight')

## Compiled decadal runoff by RCP

In [ ]:
# Make this into a visual
runoff_variables = ['melt_off_glacier_m', 'melt_on_glacier_m']
sum_vars = ['liq_prcp_off_glacier_m', 'liq_prcp_on_glacier_m']

f, ((ax1, ax2), (ax3, ax4), (ax5, ax6), (ax7, ax8)) = plt.subplots(4, 2, figsize=(7, 10), sharey=True, constrained_layout=True)

color_pr = '#946cbd'
color_m = ('#d9985f', '#d16b6c')

s_keys = rtype_calc_s26.keys()
ct = 0
for r in runoff_variables:
    runoff_s = rtype_calc_monthly(s_keys, rtype_calc_s26, r)
    runoff_e = rtype_calc_monthly(s_keys, rtype_calc_e26, r)
    xvals = index2
    ax1.plot(xvals, runoff_s['mean'], label=r, color=color_m[ct])
    ax2.plot(xvals, runoff_e['mean'], label=r, color=color_m[ct])
    ct += 1
runoff_s = rtype_calc_monthly_2vars(s_keys, rtype_calc_s26, sum_vars[0], sum_vars[1])
runoff_e = rtype_calc_monthly_2vars(s_keys, rtype_calc_e26, sum_vars[0], sum_vars[1])
xvals = index2
ax1.plot(xvals, runoff_s['mean'], label=r, color=color_pr)
ax2.plot(xvals, runoff_e['mean'], label=r, color=color_pr)

s_keys = rtype_calc_s45.keys()
ct = 0
for r in runoff_variables:
    runoff_s = rtype_calc_monthly(s_keys, rtype_calc_s45, r)
    runoff_e = rtype_calc_monthly(s_keys, rtype_calc_e45, r)
    xvals = index2
    ax3.plot(xvals, runoff_s['mean'], label=r, color=color_m[ct])
    ax4.plot(xvals, runoff_e['mean'], label=r, color=color_m[ct])
    ct += 1
runoff_s = rtype_calc_monthly_2vars(s_keys, rtype_calc_s45, sum_vars[0], sum_vars[1])
runoff_e = rtype_calc_monthly_2vars(s_keys, rtype_calc_e45, sum_vars[0], sum_vars[1])
xvals = index2
ax3.plot(xvals, runoff_s['mean'], label=r, color=color_pr)
ax4.plot(xvals, runoff_e['mean'], label=r, color=color_pr)

s_keys = rtype_calc_s60.keys()
ct = 0
for r in runoff_variables:
    runoff_s = rtype_calc_monthly(s_keys, rtype_calc_s60, r)
    runoff_e = rtype_calc_monthly(s_keys, rtype_calc_e60, r)
    xvals = index2
    ax5.plot(xvals, runoff_s['mean'], label=r, color=color_m[ct])
    ax6.plot(xvals, runoff_e['mean'], label=r, color=color_m[ct])
    ct += 1
runoff_s = rtype_calc_monthly_2vars(s_keys, rtype_calc_s60, sum_vars[0], sum_vars[1])
runoff_e = rtype_calc_monthly_2vars(s_keys, rtype_calc_e60, sum_vars[0], sum_vars[1])
xvals = index2
ax5.plot(xvals, runoff_s['mean'], label=r, color=color_pr)
ax6.plot(xvals, runoff_e['mean'], label=r, color=color_pr)

s_keys = rtype_calc_s85.keys()
ct = 0
for r in runoff_variables:
    runoff_s = rtype_calc_monthly(s_keys, rtype_calc_s85, r)
    runoff_e = rtype_calc_monthly(s_keys, rtype_calc_e85, r)
    xvals = index2
    ax7.plot(xvals, runoff_s['mean'], label=r, color=color_m[ct])
    ax8.plot(xvals, runoff_e['mean'], label=r, color=color_m[ct])
    ct += 1
runoff_s = rtype_calc_monthly_2vars(s_keys, rtype_calc_s85, sum_vars[0], sum_vars[1])
runoff_e = rtype_calc_monthly_2vars(s_keys, rtype_calc_e85, sum_vars[0], sum_vars[1])
xvals = index2
ax7.plot(xvals, runoff_s['mean'], label=r, color=color_pr)
ax8.plot(xvals, runoff_e['mean'], label=r, color=color_pr)

ax1.set(ylabel='Runoff [Mt]', title='2020-2030') # this title and the next one are overwritten by my addition
ax2.set(title='2090-2100')
ax3.set(ylabel='Runoff [Mt]')
ax5.set(ylabel='Runoff [Mt]')
ax7.set(ylabel='Runoff [Mt]', xlabel='Month')
ax8.set(xlabel='Month')

# THIS WORKS
for ax in (ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8):
    ax.set(xticks=season_roll['month_2d'].values,
           xticklabels=season_roll2['month_2d'].values,  # here, we want calendar year values
           )

# Add RCP labels as titles (from chatgpt)
# ax1.set_title('RCP 2.6: 2020-2030', fontsize=10)
# ax2.set_title('RCP 2.6: 2090-2100', fontsize=10)
# ax3.set_title('RCP 4.5: 2020-2030', fontsize=10)
# ax4.set_title('RCP 4.5: 2090-2100', fontsize=10)
# ax5.set_title('RCP 6.0: 2020-2030', fontsize=10)
# ax6.set_title('RCP 6.0: 2090-2100', fontsize=10)
# ax7.set_title('RCP 8.5: 2020-2030', fontsize=10)
# ax8.set_title('RCP 8.5: 2090-2100', fontsize=10)

ax1.text(0.04, 0.9, 'RCP 2.6', transform=ax1.transAxes)

plt.suptitle('Decadal seasonal runoff averages by runoff component', fontweight='bold', fontsize=14)

plt.show()


In [ ]:
# Visualize seasonal runoff by component for each GCM
runoff_variables = ['melt_off_glacier_m', 'melt_on_glacier_m']
sum_vars = ['liq_prcp_off_glacier_m', 'liq_prcp_on_glacier_m']

f, ((ax1, ax2), (ax3, ax4), (ax5, ax6), (ax7, ax8)) = plt.subplots(4, 2, figsize=(7, 10), sharey=True, constrained_layout = True);

color_pr = '#946cbd'
color_m = ('#d9985f', '#d16b6c')

# RCP2.6
s_keys = rtype_calc_s26.keys()
ct=0
for r in runoff_variables:
    runoff_s = rtype_calc_monthly(s_keys, rtype_calc_s26, r)
    runoff_e = rtype_calc_monthly(s_keys, rtype_calc_e26, r)
    xvals = index2
    for g in s_keys:
        ax1.plot(xvals, runoff_s[g], color=color_m[ct], linewidth=0.4)
        ax2.plot(xvals, runoff_e[g], color=color_m[ct], linewidth=0.4)
    ct=ct+1
runoff_s = rtype_calc_monthly_2vars(s_keys, rtype_calc_s26, sum_vars[0], sum_vars[1])
runoff_e = rtype_calc_monthly_2vars(s_keys, rtype_calc_e26, sum_vars[0], sum_vars[1])
for g in s_keys:
    ax1.plot(xvals, runoff_s[g], color=color_pr, linewidth=0.4)
    ax2.plot(xvals, runoff_e[g], color=color_pr, linewidth=0.4)
    

# RCP4.5
s_keys = rtype_calc_s45.keys()
ct=0
for r in runoff_variables:
    runoff_s = rtype_calc_monthly(s_keys, rtype_calc_s45, r)
    runoff_e = rtype_calc_monthly(s_keys, rtype_calc_e45, r)
    xvals = index2
    for g in s_keys:
        ax3.plot(xvals, runoff_s[g], color=color_m[ct], linewidth=0.4)
        ax4.plot(xvals, runoff_e[g], color=color_m[ct], linewidth=0.4)
    ct=ct+1
runoff_s = rtype_calc_monthly_2vars(s_keys, rtype_calc_s45, sum_vars[0], sum_vars[1])
runoff_e = rtype_calc_monthly_2vars(s_keys, rtype_calc_e45, sum_vars[0], sum_vars[1])
for g in s_keys:
    ax3.plot(xvals, runoff_s[g], color=color_pr, linewidth=0.4)
    ax4.plot(xvals, runoff_e[g], color=color_pr, linewidth=0.4)


# RCP6.0
s_keys = rtype_calc_s60.keys()
ct=0
for r in runoff_variables:
    runoff_s = rtype_calc_monthly(s_keys, rtype_calc_s60, r)
    runoff_e = rtype_calc_monthly(s_keys, rtype_calc_e60, r)
    xvals = index2
    for g in s_keys:
        ax5.plot(xvals, runoff_s[g], color=color_m[ct], linewidth=0.4)
        ax6.plot(xvals, runoff_e[g], color=color_m[ct], linewidth=0.4)
    ct=ct+1
runoff_s = rtype_calc_monthly_2vars(s_keys, rtype_calc_s60, sum_vars[0], sum_vars[1])
runoff_e = rtype_calc_monthly_2vars(s_keys, rtype_calc_e60, sum_vars[0], sum_vars[1])
for g in s_keys:
    ax5.plot(xvals, runoff_s[g], color=color_pr, linewidth=0.4)
    ax6.plot(xvals, runoff_e[g], color=color_pr, linewidth=0.4)

# RCP8.5
s_keys = rtype_calc_s85.keys()
ct=0
for r in runoff_variables:
    runoff_s = rtype_calc_monthly(s_keys, rtype_calc_s85, r)
    runoff_e = rtype_calc_monthly(s_keys, rtype_calc_e85, r)
    xvals = index2
    for g in s_keys:
        ax7.plot(xvals, runoff_s[g], color=color_m[ct], linewidth=0.4)
        ax8.plot(xvals, runoff_e[g], color=color_m[ct], linewidth=0.4)
    ct=ct+1
runoff_s = rtype_calc_monthly_2vars(s_keys, rtype_calc_s85, sum_vars[0], sum_vars[1])
runoff_e = rtype_calc_monthly_2vars(s_keys, rtype_calc_e85, sum_vars[0], sum_vars[1])
for g in s_keys:
    ax7.plot(xvals, runoff_s[g], color=color_pr, linewidth=0.4)
    ax8.plot(xvals, runoff_e[g], color=color_pr, linewidth=0.4)
    

ax1.set(ylabel='Runoff [Mt]', title='2020-2030')
ax2.set(title='2090-2100')
ax3.set(ylabel='Runoff [Mt]')
ax5.set(ylabel='Runoff [Mt]')
ax7.set(ylabel='Runoff [Mt]', xlabel = 'Month')
ax8.set(xlabel = 'Month')
# THIS WORKS
for ax in (ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8):
    ax.set(xticks = season_roll['month_2d'].values,
           xticklabels = season_roll2['month_2d'].values, #here, we want calendar year values
          );

plt.suptitle('Multi-GCM decadal seasonal runoff by runoff component', fontweight = 'bold', fontsize = 14);

#plt.savefig('Figures/Seasonal/Decade_av/4x2_seasonsall_rcps_allgcms.png', bbox_inches='tight')

### Plot of relative contribution on decade scale

In [ ]:
# Set up the dictionaries with values to use in the summing function
s_keys_dict = {}
s_keys_dict['rcp26'] = rtype_calc_s26.keys()
s_keys_dict['rcp45'] = rtype_calc_s45.keys()
s_keys_dict['rcp60'] = rtype_calc_s60.keys()
s_keys_dict['rcp85'] = rtype_calc_s85.keys()

rtype_calc_dict_s = {}
rtype_calc_dict_s['rcp26'] = rtype_calc_s26
rtype_calc_dict_s['rcp45'] = rtype_calc_s45
rtype_calc_dict_s['rcp60'] = rtype_calc_s60
rtype_calc_dict_s['rcp85'] = rtype_calc_s85

rtype_calc_dict_e = {}
rtype_calc_dict_e['rcp26'] = rtype_calc_e26
rtype_calc_dict_e['rcp45'] = rtype_calc_e45
rtype_calc_dict_e['rcp60'] = rtype_calc_e60
rtype_calc_dict_e['rcp85'] = rtype_calc_e85

In [ ]:
rcp = ('rcp26', 'rcp45', 'rcp60', 'rcp85')

all_rcp_dict_s = {}

for r in rcp:
    melt_on = rtype_calc_monthly(s_keys_dict[r], rtype_calc_dict_s[r], 'melt_on_glacier_m')
    melt_off = rtype_calc_monthly(s_keys_dict[r], rtype_calc_dict_s[r], 'melt_off_glacier_m')
    liq = rtype_calc_monthly_2vars(s_keys_dict[r], rtype_calc_dict_s[r], 'liq_prcp_off_glacier_m', 'liq_prcp_on_glacier_m')
    summed = melt_on + melt_off + liq
    m_on_name = 'm-on-{}'.format(r)
    m_off_name = 'm-off-{}'.format(r)
    liq_name = 'liq-{}'.format(r)
    sum_name = 'sum-{}'.format(r)
    all_rcp_dict_s[m_on_name] = melt_on
    all_rcp_dict_s[m_off_name] = melt_off
    all_rcp_dict_s[liq_name] = liq
    all_rcp_dict_s[sum_name] = summed

all_rcp_dict_e = {}

for r in rcp:
    melt_on = rtype_calc_monthly(s_keys_dict[r], rtype_calc_dict_e[r], 'melt_on_glacier_m')
    melt_off = rtype_calc_monthly(s_keys_dict[r], rtype_calc_dict_e[r], 'melt_off_glacier_m')
    liq = rtype_calc_monthly_2vars(s_keys_dict[r], rtype_calc_dict_e[r], 'liq_prcp_off_glacier_m', 'liq_prcp_on_glacier_m')
    summed = melt_on + melt_off + liq
    m_on_name = 'm-on-{}'.format(r)
    m_off_name = 'm-off-{}'.format(r)
    liq_name = 'liq-{}'.format(r)
    sum_name = 'sum-{}'.format(r)
    all_rcp_dict_e[m_on_name] = melt_on
    all_rcp_dict_e[m_off_name] = melt_off
    all_rcp_dict_e[liq_name] = liq
    all_rcp_dict_e[sum_name] = summed

In [ ]:
# Graph of relative importance of runoff for each part of the seasonal cycle (decade 1 and last)

f, ((ax1, ax2), (ax3, ax4), (ax5, ax6), (ax7, ax8)) = plt.subplots(4, 2, figsize=(9, 11), sharey=True, constrained_layout = True);

color_pr = '#946cbd'
color_m = ('#d9985f', '#d16b6c')

xvals = index2

# RCP 2.6
rcp_key = 'rcp26'
ax1.stackplot(xvals, all_rcp_dict_s['m-on-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_s['sum-{}'.format(rcp_key)]['mean'], 
              all_rcp_dict_s['m-off-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_s['sum-{}'.format(rcp_key)]['mean'],
              all_rcp_dict_s['liq-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_s['sum-{}'.format(rcp_key)]['mean'],
              labels=['Melt on glacier', 'Melt off glacier', 'Liquid precipitation'],
              colors=['#d16b6c', '#d9985f', '#946cbd']);

ax2.stackplot(xvals, all_rcp_dict_e['m-on-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_e['sum-{}'.format(rcp_key)]['mean'], 
              all_rcp_dict_e['m-off-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_e['sum-{}'.format(rcp_key)]['mean'],
              all_rcp_dict_e['liq-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_e['sum-{}'.format(rcp_key)]['mean'],
              labels=['Melt on glacier', 'Melt off glacier', 'Liquid precipitation'],
              colors=['#d16b6c', '#d9985f', '#946cbd']);

# RCP4.5
rcp_key = 'rcp45'
ax3.stackplot(xvals, all_rcp_dict_s['m-on-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_s['sum-{}'.format(rcp_key)]['mean'], 
              all_rcp_dict_s['m-off-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_s['sum-{}'.format(rcp_key)]['mean'],
              all_rcp_dict_s['liq-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_s['sum-{}'.format(rcp_key)]['mean'],
              labels=['Melt on glacier', 'Melt off glacier', 'Liquid precipitation'],
              colors=['#d16b6c', '#d9985f', '#946cbd']);

ax4.stackplot(xvals, all_rcp_dict_e['m-on-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_e['sum-{}'.format(rcp_key)]['mean'], 
              all_rcp_dict_e['m-off-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_e['sum-{}'.format(rcp_key)]['mean'],
              all_rcp_dict_e['liq-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_e['sum-{}'.format(rcp_key)]['mean'],
              labels=['Melt on glacier', 'Melt off glacier', 'Liquid precipitation'],
              colors=['#d16b6c', '#d9985f', '#946cbd']);

# RCP6.0
rcp_key = 'rcp60'
ax5.stackplot(xvals, all_rcp_dict_s['m-on-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_s['sum-{}'.format(rcp_key)]['mean'], 
              all_rcp_dict_s['m-off-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_s['sum-{}'.format(rcp_key)]['mean'],
              all_rcp_dict_s['liq-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_s['sum-{}'.format(rcp_key)]['mean'],
              labels=['Melt on glacier', 'Melt off glacier', 'Liquid precipitation'],
              colors=['#d16b6c', '#d9985f', '#946cbd']);

ax6.stackplot(xvals, all_rcp_dict_e['m-on-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_e['sum-{}'.format(rcp_key)]['mean'], 
              all_rcp_dict_e['m-off-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_e['sum-{}'.format(rcp_key)]['mean'],
              all_rcp_dict_e['liq-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_e['sum-{}'.format(rcp_key)]['mean'],
              labels=['Melt on glacier', 'Melt off glacier', 'Liquid precipitation'],
              colors=['#d16b6c', '#d9985f', '#946cbd']);

# RCP85
rcp_key = 'rcp85'
ax7.stackplot(xvals, all_rcp_dict_s['m-on-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_s['sum-{}'.format(rcp_key)]['mean'], 
              all_rcp_dict_s['m-off-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_s['sum-{}'.format(rcp_key)]['mean'],
              all_rcp_dict_s['liq-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_s['sum-{}'.format(rcp_key)]['mean'],
              labels=['Melt on glacier', 'Melt off glacier', 'Liquid precipitation'],
              colors=['#d16b6c', '#d9985f', '#946cbd']);

ax8.stackplot(xvals, all_rcp_dict_e['m-on-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_e['sum-{}'.format(rcp_key)]['mean'], 
              all_rcp_dict_e['m-off-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_e['sum-{}'.format(rcp_key)]['mean'],
              all_rcp_dict_e['liq-{}'.format(rcp_key)]['mean']*100/all_rcp_dict_e['sum-{}'.format(rcp_key)]['mean'],
              labels=['Melt on glacier', 'Melt off glacier', 'Liquid precipitation'],
              colors=['#d16b6c', '#d9985f', '#946cbd']);

#ax1.legend(bbox_to_anchor=(0.44, 0.95));

plt.suptitle('Relative seasonal contribution of runoff components', fontweight = 'bold', fontsize = 14);

ax1.set(ylabel='% Total runoff', title='2020-2030')
ax2.set(title='2090-2100')
ax3.set(ylabel='% Total runoff')
ax5.set(ylabel='% Total runoff')
ax7.set(ylabel='% Total runoff', xlabel = 'Month')
ax8.set(xlabel = 'Month')
# THIS WORKS
for ax in (ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8):
    ax.set(xticks = season_roll['month_2d'].values,
           xticklabels = season_roll2['month_2d'].values #here, we want calendar year values
          );
# Add RCP labels as titles (from chatgpt)
ax1.set_title('RCP 2.6: 2020-2030', fontsize=10)
ax2.set_title('RCP 2.6: 2090-2100', fontsize=10)
ax3.set_title('RCP 4.5: 2020-2030', fontsize=10)
ax4.set_title('RCP 4.5: 2090-2100', fontsize=10)
ax5.set_title('RCP 6.0: 2020-2030', fontsize=10)
ax6.set_title('RCP 6.0: 2090-2100', fontsize=10)
ax7.set_title('RCP 8.5: 2020-2030', fontsize=10)
ax8.set_title('RCP 8.5: 2090-2100', fontsize=10)

#plt.savefig('Figures/Seasonal/Decade_av/4x2_relative_importance_newMay4.png', bbox_inches='tight')

# Step 4: plot annual runoff

## Plotting

In [ ]:
import xarray as xr

# Function to calculate annual runoff
def calc_annual_runoff(data_array):
    # Sum over the month_2d dimension to get annual data
    annual_sum = data_array.sum(dim='month_2d')
    return annual_sum

# Dictionary to store annual runoff for each GCM and RCP scenario
annual_runoff_dict = {}

# List of scenarios
scenarios = ['NorESM1-M-rcp26-calc', 'CCSM4-rcp26-calc']

for scenario in scenarios:
    if scenario in season_unroll26:
        data_array = season_unroll26[scenario]
        annual_runoff = calc_annual_runoff(data_array)
        annual_runoff_dict[scenario] = annual_runoff
    else:
        print(f"Scenario '{scenario}' not found in season_unroll26")


In [ ]:
import matplotlib.pyplot as plt

# Function to plot annual runoff
def plot_annual_runoff(annual_runoff_dict):
    plt.figure(figsize=(10, 6))

    for scenario, annual_runoff in annual_runoff_dict.items():
        years = annual_runoff['time'].values
        runoff_values = annual_runoff.values
        plt.plot(years, runoff_values, label=scenario)
    
    plt.xlabel('Year')
    plt.ylabel('Annual Runoff (10^9 m^3)')
    plt.title('Annual Runoff for Different GCM and RCP Scenarios')
    plt.legend()
    plt.show()

# Call the plotting function
plot_annual_runoff(annual_runoff_dict)


In [ ]:
import xarray as xr

# Function to calculate annual runoff
def calc_annual_runoff(data_array):
    # Sum over the month_2d dimension to get annual data
    annual_sum = data_array.sum(dim='month_2d')
    return annual_sum

# Dictionary to store annual runoff for each GCM and RCP scenario
annual_runoff_dict = {}

# List of scenarios and datasets
scenarios_datasets = {
    'NorESM1-M-rcp26-calc': season_unroll26['NorESM1-M-rcp26-calc'],
    'CCSM4-rcp26-calc': season_unroll26['CCSM4-rcp26-calc'],
    'NorESM1-M-rcp45-calc': season_unroll45['NorESM1-M-rcp45-calc'],
    'CCSM4-rcp45-calc': season_unroll45['CCSM4-rcp45-calc'],
    'NorESM1-M-rcp60-calc': season_unroll60['NorESM1-M-rcp60-calc'],
    'CCSM4-rcp60-calc': season_unroll60['CCSM4-rcp60-calc'],
    'NorESM1-M-rcp85-calc': season_unroll85['NorESM1-M-rcp85-calc'],
    'CCSM4-rcp85-calc': season_unroll85['CCSM4-rcp85-calc']
}

for scenario, data_array in scenarios_datasets.items():
    annual_runoff = calc_annual_runoff(data_array)
    annual_runoff_dict[scenario] = annual_runoff

#look at the data array for annual runoff?

In [ ]:
print(annual_runoff)

In [ ]:
import matplotlib.pyplot as plt

# Function to plot annual runoff
def plot_annual_runoff(annual_runoff_dict):
    plt.figure(figsize=(14, 8))

    for scenario, annual_runoff in annual_runoff_dict.items():
        years = annual_runoff['time'].values
        runoff_values = annual_runoff.values
        plt.plot(years, runoff_values, label=scenario)
    
    plt.xlabel('Year')
    plt.ylabel('Annual Runoff (10^9 m^3)')
    plt.title('Annual Runoff for Different GCM and RCP Scenarios')
    plt.legend(loc='upper right', bbox_to_anchor=(1.15, 1))
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Call the plotting function
plot_annual_runoff(annual_runoff_dict)


In [ ]:
import xarray as xr
import matplotlib.pyplot as plt

# Function to calculate annual runoff
def calc_annual_runoff(data_array):
    # Sum over the month_2d dimension to get annual data
    annual_sum = data_array.sum(dim='month_2d')
    return annual_sum

# Dictionary to store annual runoff for each GCM and RCP scenario
annual_runoff_dict = {}

# List of scenarios and datasets
scenarios_datasets = {
    'NorESM1-M-rcp26-calc': season_unroll26['NorESM1-M-rcp26-calc'],
    'CCSM4-rcp26-calc': season_unroll26['CCSM4-rcp26-calc'],
    'NorESM1-M-rcp45-calc': season_unroll45['NorESM1-M-rcp45-calc'],
    'CCSM4-rcp45-calc': season_unroll45['CCSM4-rcp45-calc'],
    'NorESM1-M-rcp60-calc': season_unroll60['NorESM1-M-rcp60-calc'],
    'CCSM4-rcp60-calc': season_unroll60['CCSM4-rcp60-calc'],
    'NorESM1-M-rcp85-calc': season_unroll85['NorESM1-M-rcp85-calc'],
    'CCSM4-rcp85-calc': season_unroll85['CCSM4-rcp85-calc']
}

# Calculate annual runoff for each scenario
for scenario, data_array in scenarios_datasets.items():
    annual_runoff = calc_annual_runoff(data_array)
    annual_runoff_dict[scenario] = annual_runoff

# List of RCP scenarios
rcps = ['rcp26', 'rcp45', 'rcp60', 'rcp85']

# Function to plot annual runoff for each RCP scenario
def plot_rcp_separate(annual_runoff_dict, rcps):
    fig, axes = plt.subplots(len(rcps), 1, figsize=(14, 24), sharex=True)
    
    for i, rcp in enumerate(rcps):
        ax = axes[i]
        ax.set_title(f'Annual Runoff for RCP {rcp}')
        
        for scenario, annual_runoff in annual_runoff_dict.items():
            if rcp in scenario:
                years = annual_runoff['time'].values
                runoff_values = annual_runoff.values
                ax.plot(years, runoff_values, label=scenario)
        
        ax.set_xlabel('Year')
        ax.set_ylabel('Annual Runoff (10^9 m^3)')
        ax.legend(loc='upper right', bbox_to_anchor=(1.15, 1))
        ax.grid(True)
    
    plt.tight_layout()
    plt.show()

# Call the plotting function
plot_rcp_separate(annual_runoff_dict, rcps)


In [ ]:
import xarray as xr
import matplotlib.pyplot as plt

# Function to calculate annual runoff
def calc_annual_runoff(data_array):
    # Sum over the month_2d dimension to get annual data
    annual_sum = data_array.sum(dim='month_2d')
    return annual_sum

# Function to calculate the mean annual runoff across GCMs for each RCP scenario
def calculate_mean_annual_runoff(annual_runoff_dict, rcps):
    mean_annual_runoff = {}
    for rcp in rcps:
        rcp_data = [annual_runoff for scenario, annual_runoff in annual_runoff_dict.items() if rcp in scenario]
        if rcp_data:
            combined = xr.concat(rcp_data, dim='gcm')
            mean_annual_runoff[rcp] = combined.mean(dim='gcm')
    return mean_annual_runoff

# Function to plot the mean annual runoff for each RCP scenario on one plot
def plot_mean_annual_runoff(mean_annual_runoff):
    plt.figure(figsize=(14, 8))

    for rcp, annual_runoff in mean_annual_runoff.items():
        years = annual_runoff['time'].values
        runoff_values = annual_runoff.values
        plt.plot(years, runoff_values, label=f'{rcp} Mean')
    
    plt.xlabel('Year')
    plt.ylabel('Annual Runoff (10^9 m^3)')
    plt.title('Mean Annual Runoff for Different RCP Scenarios')
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Dictionary to store annual runoff for each GCM and RCP scenario
annual_runoff_dict = {
    'NorESM1-M-rcp26-calc': calc_annual_runoff(season_unroll26['NorESM1-M-rcp26-calc']), #calc_annual_runoff(melt_on_glacier_monthly['NorESM1-M-rcp26-calc']
    'CCSM4-rcp26-calc': calc_annual_runoff(season_unroll26['CCSM4-rcp26-calc']),
    'NorESM1-M-rcp45-calc': calc_annual_runoff(season_unroll45['NorESM1-M-rcp45-calc']),
    'CCSM4-rcp45-calc': calc_annual_runoff(season_unroll45['CCSM4-rcp45-calc']),
    'NorESM1-M-rcp60-calc': calc_annual_runoff(season_unroll60['NorESM1-M-rcp60-calc']),
    'CCSM4-rcp60-calc': calc_annual_runoff(season_unroll60['CCSM4-rcp60-calc']),
    'NorESM1-M-rcp85-calc': calc_annual_runoff(season_unroll85['NorESM1-M-rcp85-calc']),
    'CCSM4-rcp85-calc': calc_annual_runoff(season_unroll85['CCSM4-rcp85-calc'])
}

# List of RCP scenarios
rcps = ['rcp26', 'rcp45', 'rcp60', 'rcp85']

# Calculate the mean annual runoff for each RCP scenario
mean_annual_runoff = calculate_mean_annual_runoff(annual_runoff_dict, rcps)

# Call the plotting function
plot_mean_annual_runoff(mean_annual_runoff)

## trying to plot runoff components (unsuccessful)

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt

# Function to calculate annual runoff for each component
def calc_annual_runoff_components(data_array):
    # Sum over the month_2d dimension to get annual data for each component
    annual_melt_on_glacier = data_array['melt_on_glacier_monthly'].sum(dim='month_2d')
    annual_melt_off_glacier = data_array['melt_off_glacier_monthly'].sum(dim='month_2d')
    annual_liq_prcp_off_glacier = data_array['liq_prcp_off_glacier_monthly'].sum(dim='month_2d')
    annual_liq_prcp_on_glacier = data_array['liq_prcp_on_glacier_monthly'].sum(dim='month_2d')
    
    # Convert units from m^3 to 10^9 m^3
    annual_melt_on_glacier *= 1e-9
    annual_melt_off_glacier *= 1e-9
    annual_liq_prcp_off_glacier *= 1e-9
    annual_liq_prcp_on_glacier *= 1e-9
    
    return annual_melt_on_glacier, annual_melt_off_glacier, annual_liq_prcp_off_glacier, annual_liq_prcp_on_glacier

# Function to calculate the mean annual runoff across GCMs for each RCP scenario
def calculate_mean_annual_runoff(annual_runoff_dict, rcps):
    mean_annual_runoff = {}
    for rcp in rcps:
        rcp_data = [annual_runoff for scenario, annual_runoff in annual_runoff_dict.items() if rcp in scenario]
        if rcp_data:
            combined = xr.concat(rcp_data, dim='gcm')
            mean_annual_runoff[rcp] = combined.mean(dim='gcm')
    return mean_annual_runoff

# Function to plot the mean annual runoff for each RCP scenario on one plot
def plot_mean_annual_runoff(mean_annual_runoff, component_name):
    plt.figure(figsize=(14, 8))

    for rcp, annual_runoff in mean_annual_runoff.items():
        years = annual_runoff['time'].dt.year.values
        runoff_values = annual_runoff.values
        plt.plot(years, runoff_values, label=f'{rcp} Mean')
    
    plt.xlabel('Year')
    plt.ylabel('Annual Runoff (10^9 m^3)')
    plt.title(f'Mean Annual {component_name} Runoff for Different RCP Scenarios')
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Load the dataset
this_ds = xr.open_dataset('path_to_your_file')

# Calculate annual runoff for each component for each scenario
annual_runoff_dict = {
    'NorESM1-M-rcp26-calc': calc_annual_runoff_components(this_ds['NorESM1-M-rcp26-calc']),
    'CCSM4-rcp26-calc': calc_annual_runoff_components(this_ds['CCSM4-rcp26-calc']),
    'NorESM1-M-rcp45-calc': calc_annual_runoff_components(this_ds['NorESM1-M-rcp45-calc']),
    'CCSM4-rcp45-calc': calc_annual_runoff_components(this_ds['CCSM4-rcp45-calc']),
    'NorESM1-M-rcp60-calc': calc_annual_runoff_components(this_ds['NorESM1-M-rcp60-calc']),
    'CCSM4-rcp60-calc': calc_annual_runoff_components(this_ds['CCSM4-rcp60-calc']),
    'NorESM1-M-rcp85-calc': calc_annual_runoff_components(this_ds['NorESM1-M-rcp85-calc']),
    'CCSM4-rcp85-calc': calc_annual_runoff_components(this_ds['CCSM4-rcp85-calc'])
}

# List of RCP scenarios
rcps = ['rcp26', 'rcp45', 'rcp60', 'rcp85']

# Calculate and plot for each runoff component
component_names = ['Melt On Glacier', 'Melt Off Glacier', 'Liquid Precipitation Off Glacier', 'Liquid Precipitation On Glacier']

for i, component_name in enumerate(component_names):
    annual_runoff_dict_component = {key: val[i] for key, val in annual_runoff_dict.items()}
    mean_annual_runoff_component = calculate_mean_annual_runoff(annual_runoff_dict_component, rcps)
    plot_mean_annual_runoff(mean_annual_runoff_component, component_name)


In [ ]:

# Function: load the dataset
def open_series(fpath_stem, which_date, which_gcm, which_rcp):
    this_fpath = fpath_stem+'{}-gcm_data_{}_{}_compiled_output.nc'.format(which_date, which_gcm, which_rcp)
    this_ds = xr.open_dataset(this_fpath)
    return this_ds

# Define file path and parameters
fpath_stem = '/Users/ziggygoddard/Documents/summer24/'
which_date = '2024-06-05'
which_gcm = 'NorESM1-M'
which_rcp = 'rcp26'

# Load the dataset
this_ds = open_series(fpath_stem, which_date, which_gcm, which_rcp)

# Verify the dataset
print(this_ds)

# Calculate runoff (as an example, not necessary to run this block for verification)
def calc_seasonal(this_ds):
    melt1 = this_ds.melt_on_glacier_monthly
    melt2 = this_ds.melt_off_glacier_monthly
    melt3 = this_ds.liq_prcp_off_glacier_monthly
    melt4 = this_ds.liq_prcp_on_glacier_monthly
    
    this_sum = melt1.sum(dim='rgi_id').dropna(dim='time') + melt2.sum(dim='rgi_id').dropna(dim='time') + melt3.sum(dim='rgi_id').dropna(dim='time') + melt4.sum(dim='rgi_id').dropna(dim='time')
    nonzero_sum = this_sum.where(this_sum!=0, drop=True)
    nonzero_sum *= 1e-9

    return nonzero_sum


In [ ]:
# This step should create plottable annual values of the 4 runoff components

melt1 = this_ds.melt_on_glacier_monthly
melt2 = this_ds.melt_off_glacier_monthly
melt3 = this_ds.liq_prcp_off_glacier_monthly
melt4 = this_ds.liq_prcp_on_glacier_monthly

this_sum1 = melt1.sum(dim='rgi_id').dropna(dim='time')
this_sum2 = melt2.sum(dim='rgi_id').dropna(dim='time')
this_sum3 = melt3.sum(dim='rgi_id').dropna(dim='time')
this_sum4 = melt4.sum(dim='rgi_id').dropna(dim='time')

# Filter out zero values and convert units
annual_melt_on_glacier = this_sum1.where(this_sum1 != 0, drop=True) * 1e-9
annual_melt_off_glacier = this_sum2.where(this_sum2 != 0, drop=True) * 1e-9
annual_liq_prcp_off_glacier = this_sum3.where(this_sum3 != 0, drop=True) * 1e-9
annual_liq_prcp_on_glacier = this_sum4.where(this_sum4 != 0, drop=True) * 1e-9

In [ ]:
# Dictionary to store annual runoff for each GCM and RCP scenario
annual_runoff_dict = {
    'melt_on_glacier': annual_melt_on_glacier,
    'melt_off_glacier': annual_melt_off_glacier,
    'liq_prcp_off_glacier': annual_liq_prcp_off_glacier,
    'liq_prcp_on_glacier': annual_liq_prcp_on_glacier
}

# Calculate the mean annual runoff for each scenario
mean_annual_runoff = calculate_mean_annual_runoff(annual_runoff_dict, rcps)

# Call the plotting function
plot_mean_annual_runoff(mean_annual_runoff)

In [ ]:
print(mean_annual_runoff)

In [ ]:
def open_series(fpath_stem, date, gcm, rcp):
    # Construct the file path
    this_fpath = os.path.join(fpath_stem, f'{date}-gcm_data_{gcm}_{rcp}_compiled_output.nc')
    
    # Load the dataset
    this_ds = xr.open_dataset(this_fpath)
    
    return this_ds

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import os

# Function to calculate annual runoff
def calc_annual_runoff(data_array):
    # Sum over the time dimension to get annual data
    annual_sum = data_array.sum(dim='time')
    return annual_sum

# Function to calculate the mean annual runoff across GCMs for each RCP scenario
def calculate_mean_annual_runoff(annual_runoff_dict, rcps):
    mean_annual_runoff = {}
    for rcp in rcps:
        rcp_data = [annual_runoff for scenario, annual_runoff in annual_runoff_dict.items() if rcp in scenario]
        if rcp_data:
            combined = xr.concat(rcp_data, dim='gcm')
            mean_annual_runoff[rcp] = combined.mean(dim='gcm')
    return mean_annual_runoff

# Function to plot the mean annual runoff for each RCP scenario on one plot
def plot_mean_annual_runoff(mean_annual_runoff):
    plt.figure(figsize=(14, 8))

    for rcp, annual_runoff in mean_annual_runoff.items():
        years = annual_runoff['time'].values
        runoff_values = annual_runoff.values
        plt.plot(years, runoff_values, label=f'{rcp} Mean')
    
    plt.xlabel('Year')
    plt.ylabel('Annual Runoff (10^9 m^3)')
    plt.title('Mean Annual Runoff for Different RCP Scenarios')
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Define variables for file path construction
fpath_stem = '/Users/ziggygoddard/Documents/summer24/'
date_list = ['2024-06-05', '2024-06-06']  # Example export dates
gcm_list = ['NorESM1-M', 'CCSM4']
rcp_list = ['rcp26', 'rcp45', 'rcp60', 'rcp85']

# Load the dataset and calculate annual runoff for each component for each scenario
annual_runoff_dict = {}
for gcm in gcm_list:
    for rcp in rcp_list:
        for date in date_list:
            try:
                this_ds = open_series(fpath_stem, date, gcm, rcp)
                key = f'{gcm}-{rcp}'
                annual_runoff_dict[key] = calc_annual_runoff(this_ds)
            except FileNotFoundError:
                pass

# List of RCP scenarios
rcps = ['rcp26', 'rcp45', 'rcp60', 'rcp85']

# Calculate the mean annual runoff for each RCP scenario
mean_annual_runoff = calculate_mean_annual_runoff(annual_runoff_dict, rcps)

# Call the plotting function
plot_mean_annual_runoff(mean_annual_runoff)


In [ ]:
import matplotlib.pyplot as plt

# Function to calculate the relative contribution of runoff components
def calculate_relative_contribution(all_rcp_dict, key, time_slices):
    rel_contributions = {}
    for rcp_key in ['rcp26', 'rcp45', 'rcp60', 'rcp85']:
        contributions = []
        for component in ['m-on-', 'm-off-', 'liq-']:
            contributions.append(all_rcp_dict[component + rcp_key]['mean'] * 100 / all_rcp_dict['sum-' + rcp_key]['mean'])
        rel_contributions[rcp_key] = contributions
    return rel_contributions

# Function to plot the relative contribution of runoff components for each RCP scenario
def plot_relative_contribution(rel_contributions, time_slices):
    colors = ['#d16b6c', '#d9985f', '#946cbd']
    labels = ['Melt on glacier', 'Melt off glacier', 'Liquid precipitation']
    xvals = range(1, 13)  # Assuming 12 months in a year

    fig, axs = plt.subplots(len(time_slices), 4, figsize=(16, 12), sharex=True, sharey=True)
    fig.subplots_adjust(hspace=0.4)
    
    for i, time_slice in enumerate(time_slices):
        for j, ax in enumerate(axs[i]):
            for k, rcp_key in enumerate(['rcp26', 'rcp45', 'rcp60', 'rcp85']):
                ax.stackplot(xvals, rel_contributions[rcp_key][k], labels=labels, colors=colors)
            ax.set_title(time_slice)
            ax.legend(loc='upper left')
            ax.set_xlabel('Month')
            ax.set_ylabel('% Total runoff')

    plt.suptitle('Relative annual contribution of runoff components for each RCP scenario', fontweight='bold', fontsize=14)
    plt.show()

# Example data
all_rcp_dict_s = {}  # Assume you have a dictionary like this with data
all_rcp_dict_e = {}  # Assume you have a dictionary like this with data
index2 = range(1, 13)  # Assume you have an index like this for months
time_slices = ['2020-2030', '2090-2100']

# Calculate relative contribution of runoff components
rel_contributions_s = calculate_relative_contribution(all_rcp_dict_s, 'mean', time_slices)
rel_contributions_e = calculate_relative_contribution(all_rcp_dict_e, 'mean', time_slices)

# Plot relative contribution of runoff components
plot_relative_contribution(rel_contributions_s, time_slices)
plot_relative_contribution(rel_contributions_e, time_slices)


## success

In [ ]:
# Import packages
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import pandas as pd
import seaborn as sns
import os

In [ ]:
# Function: load the dataset
def open_series(fpath_stem, which_date, which_gcm, which_rcp):
    this_fpath = fpath_stem + '{}-gcm_data_{}_{}_compiled_output.nc'.format(which_date, which_gcm, which_rcp)
    if os.path.exists(this_fpath):
        this_ds = xr.open_dataset(this_fpath)
        return this_ds
    else:
        return None

In [ ]:
# Function to calculate annual runoff for a single component
def calc_annual_runoff(data_array):
    # Sum over the month_2d dimension to get annual data
    annual_sum = data_array.sum(dim='month_2d')
    return annual_sum

In [ ]:
# # Commented-out code doesn't sum precip, still useful to see
# # Function to calculate annual runoff components
# def calc_annual_runoff_components(this_ds):
#     components = {
#         'melt_on_glacier': calc_annual_runoff(this_ds.melt_on_glacier_monthly),
#         'melt_off_glacier': calc_annual_runoff(this_ds.melt_off_glacier_monthly),
#         'liq_prcp_off_glacier': calc_annual_runoff(this_ds.liq_prcp_off_glacier_monthly),
#         'liq_prcp_on_glacier': calc_annual_runoff(this_ds.liq_prcp_on_glacier_monthly)
#     }
#     # Convert the units to 10^9 m^3
#     for key in components:
#         components[key] = components[key].sum(dim='rgi_id').dropna(dim='time') * 1e-9
#     return components

# Function to calculate annual runoff components
def calc_annual_runoff_components(this_ds):
    melt_on_glacier = this_ds.melt_on_glacier_monthly.sum(dim='month_2d').sum(dim='rgi_id').dropna(dim='time')
    melt_off_glacier = this_ds.melt_off_glacier_monthly.sum(dim='month_2d').sum(dim='rgi_id').dropna(dim='time')
    liq_prcp_off_glacier = this_ds.liq_prcp_off_glacier_monthly.sum(dim='month_2d').sum(dim='rgi_id').dropna(dim='time')
    liq_prcp_on_glacier = this_ds.liq_prcp_on_glacier_monthly.sum(dim='month_2d').sum(dim='rgi_id').dropna(dim='time')

    annual_liq_prcp = liq_prcp_off_glacier + liq_prcp_on_glacier

    melt_on_glacier *= 1e-9
    melt_off_glacier *= 1e-9
    annual_liq_prcp *= 1e-9

    return {
        'melt_on_glacier': melt_on_glacier,
        'melt_off_glacier': melt_off_glacier,
        'annual_liq_prcp': annual_liq_prcp
    }

In [ ]:
# Define the file path, dates, GCMs, and RCPs
fpath_stem = '/Users/ziggygoddard/Documents/summer24/'
date_list = ['2024-06-05', '2024-06-06']  # Example export dates
gcm_list = ['NorESM1-M', 'CCSM4']
rcp_list = ['rcp26', 'rcp45', 'rcp60', 'rcp85']

In [ ]:
# Dictionary to store annual runoff for each GCM and RCP scenario
annual_runoff_dict = {}
for gcm in gcm_list:
    for rcp in rcp_list:
        for date in date_list:
            this_ds = open_series(fpath_stem, date, gcm, rcp)
            if this_ds:
                key = f'{gcm}-{rcp}'
                annual_runoff_dict[key] = calc_annual_runoff_components(this_ds)

In [ ]:
# # This commented out code doesn't sum prcp
# # Function to calculate the mean annual runoff across GCMs for each RCP scenario
# def calculate_mean_annual_runoff(annual_runoff_dict, rcps):
#     mean_annual_runoff = {}
#     for rcp in rcps:
#         mean_annual_runoff[rcp] = {}
#         for component in annual_runoff_dict[next(iter(annual_runoff_dict))].keys():
#             rcp_data = [annual_runoff[component] for scenario, annual_runoff in annual_runoff_dict.items() if rcp in scenario]
#             if rcp_data:
#                 combined = xr.concat(rcp_data, dim='gcm')
#                 mean_annual_runoff[rcp][component] = combined.mean(dim='gcm')
#     return mean_annual_runoff

# Function to calculate the mean annual runoff across GCMs for each RCP scenario
def calculate_mean_annual_runoff(annual_runoff_dict, rcps):
    mean_annual_runoff = {}
    for rcp in rcps:
        rcp_data = [annual_runoff for scenario, annual_runoff in annual_runoff_dict.items() if rcp in scenario]
        if rcp_data:
            combined = {component: xr.concat([data[component] for data in rcp_data], dim='gcm').mean(dim='gcm')
                        for component in rcp_data[0]}
            mean_annual_runoff[rcp] = combined
    return mean_annual_runoff

In [ ]:
# Function to plot the mean annual runoff for each RCP scenario on one plot
def plot_mean_annual_runoff(mean_annual_runoff):
    components = list(mean_annual_runoff[next(iter(mean_annual_runoff))].keys())
    
    for component in components:
        plt.figure(figsize=(14, 8))

        for rcp, annual_runoff in mean_annual_runoff.items():
            years = annual_runoff[component]['time'].values
            runoff_values = annual_runoff[component].values
            plt.plot(years, runoff_values, label=f'{rcp} Mean')
        
        plt.xlabel('Year')
        plt.ylabel(f'Annual {component.replace("_", " ").title()} (10^9 m^3)')
        plt.title(f'Mean Annual {component.replace("_", " ").title()} for Different RCP Scenarios')
        plt.legend(loc='upper right')
        plt.grid(True)
        plt.tight_layout()
        plt.show()

# # Define the file path, dates, GCMs, and RCPs
# fpath_stem = '/Users/ziggygoddard/Documents/summer24/'
# date_list = ['2024-06-05', '2024-06-06']  # Example export dates
# gcm_list = ['NorESM1-M', 'CCSM4']
# rcp_list = ['rcp26', 'rcp45', 'rcp60', 'rcp85']

# # Dictionary to store annual runoff for each GCM and RCP scenario
# annual_runoff_dict = {}
# for gcm in gcm_list:
#     for rcp in rcp_list:
#         for date in date_list:
#             this_ds = open_series(fpath_stem, date, gcm, rcp)
#             if this_ds:
#                 key = f'{gcm}-{rcp}'
#                 annual_runoff_dict[key] = calc_annual_runoff_components(this_ds)

# List of RCP scenarios
rcps = ['rcp26', 'rcp45', 'rcp60', 'rcp85']

# Calculate the mean annual runoff for each RCP scenario
mean_annual_runoff = calculate_mean_annual_runoff(annual_runoff_dict, rcps)

# Call the plotting function
plot_mean_annual_runoff(mean_annual_runoff)

In [ ]:
# Function to plot the mean annual runoff for each RCP scenario with lines for each runoff component
def plot_mean_annual_runoff(mean_annual_runoff):
    for rcp, annual_runoff in mean_annual_runoff.items():
        plt.figure(figsize=(14, 8))

        for component, data in annual_runoff.items():
            years = data['time'].values
            runoff_values = data.values
            plt.plot(years, runoff_values, label=f'{component.replace("_", " ").title()}')
        
        plt.xlabel('Year')
        plt.ylabel('Annual Runoff (10^9 m^3)')
        plt.title(f'Mean Annual Runoff Components for {rcp}')
        plt.legend(loc='upper right')
        plt.grid(True)
        plt.tight_layout()
        plt.show()

# # Define the file path, dates, GCMs, and RCPs
# fpath_stem = '/Users/ziggygoddard/Documents/summer24/'
# date_list = ['2024-06-05', '2024-06-06']  # Example export dates
# gcm_list = ['NorESM1-M', 'CCSM4']
# rcp_list = ['rcp26', 'rcp45', 'rcp60', 'rcp85']

# # Dictionary to store annual runoff for each GCM and RCP scenario
# annual_runoff_dict = {}
# for gcm in gcm_list:
#     for rcp in rcp_list:
#         for date in date_list:
#             this_ds = open_series(fpath_stem, date, gcm, rcp)
#             if this_ds:
#                 key = f'{gcm}-{rcp}'
#                 annual_runoff_dict[key] = calc_annual_runoff_components(this_ds)

# List of RCP scenarios
rcps = ['rcp26', 'rcp45', 'rcp60', 'rcp85']

# Calculate the mean annual runoff for each RCP scenario
mean_annual_runoff = calculate_mean_annual_runoff(annual_runoff_dict, rcps)

# Call the plotting function
plot_mean_annual_runoff(mean_annual_runoff)